In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 이거 실행하고 다시 런타임 시작
# 한글 폰트
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf


Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-nanum is already the newest version (20170925-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 10 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/var/cache/fontconfig: cleaning cache directory
/root/.cache/fontconfig: not cleaning non-existent 

# 라이브러리 & 데이터 불러오기

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from tqdm import tqdm
import warnings
import plotly.express as px
from matplotlib.colors import LinearSegmentedColormap
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, StratifiedKFold
sns.set(font='NaNumBarunGothic')



%matplotlib inline


pd.options.display.max_columns = None # 모든 컬럼 볼 수 있게끔
# 경고 끄기
warnings.filterwarnings(action='ignore')

In [ ]:
# 데이터 불러오기

# 기업개요 불러오기
active = pd.read_excel('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/액티브 중소법인 재무보유.xlsx')
# active는 컬럼명이 NM1 까지 붙어있어서 concat이 이상하게 된다. 그래서 컬럼명 변경~
active.rename(columns = {'CMP_NM1':'CMP_NM'}, inplace=True)

active2 = pd.read_excel('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/액티브 중소법인 재무보유.xlsx', sheet_name = '기업개요')

closing = pd.read_excel('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/휴폐업 중소법인 재무보유.xlsx')

# 휴폐업이력 불러오기
active_important = pd.read_excel('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/액티브 중소법인 재무보유.xlsx',sheet_name = '휴폐업이력')
closing_important = pd.read_excel('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/휴폐업 중소법인 재무보유.xlsx',sheet_name = '휴폐업이력')

# # 재무제표 불러오기
# fs = pd.read_table('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/재무데이터.txt', encoding = 'cp949')



# 재무제표 가공


In [ ]:
# 재무제표 불러오기
fs = pd.read_table('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/재무데이터.txt', encoding = 'cp949')


# 재무제표 채우기
from sklearn.impute import KNNImputer

def fill_fs(fs): # KNN_Imputer로 1차 채우기
  # step1. KNN_imputer - 재무제표  가중치 값으로 결측치 채우기 
  imputer = KNNImputer(n_neighbors = 2)

  knn_imputer_df = pd.DataFrame()
  for i in list(fs['사업자번호'].unique()):
    tmp = fs[fs['사업자번호']==i]
    tmp_filled = imputer.fit_transform(tmp)
    tmp_filled = pd.DataFrame(tmp_filled)
    knn_imputer_df = pd.concat([knn_imputer_df, tmp_filled], axis=0)

  return knn_imputer_df



knn_fill_df = fill_fs(fs)
knn_fill_df.columns = fs.columns
knn_fill_df['사업자번호'] = knn_fill_df['사업자번호'].astype('int')
knn_fill_df['결산년월'] = knn_fill_df['결산년월'].astype('int')

In [ ]:
# 최적의 클래스터 k 개수 구하기 위해 실루엣, 데이비스 점수 비교
# k를 2에서 10까지 돌리면서 점수 비교 후 최적의 k 개수 정하기


from sklearn.cluster import KMeans
import scipy.stats as ss

# evaluation
from sklearn.metrics.cluster import silhouette_score
from sklearn.model_selection import cross_val_score

from sklearn import metrics
from sklearn.metrics import *

cluster_df = knn_fill_df[['유동자산', '매출채권', '비유동자산']]
for i in cluster_df.columns:
  cluster_df[i] = ss.zscore(cluster_df[i])


for k in range(2, 15):
  cluster_df_copy = cluster_df.copy()
  model = KMeans(n_clusters=k)

  model.fit(cluster_df_copy)
  cluster_df_copy['cluster'] = model.labels_

  print(f'cluster k 개수 : {k}')
  print('Silhouette Coefficient: {:.4f}'.format(metrics.silhouette_score(cluster_df_copy.iloc[:,:-1], cluster_df_copy['cluster'])))
  print('Davies Bouldin Index: {:.4f}'.format(metrics.davies_bouldin_score(cluster_df_copy.iloc[:,:-1], cluster_df_copy['cluster'])))

  
# 평가는  Silhouette Coefficient, Davies bouldin score로 진행하였다.
# Silhouette Coefficient는 실루엣 계수로 -1 부터 1사이의 값을 가지며 1에 가까울 수록 최적화된 군집이라고 할 수 있다.
# Davies Bouldin Index는 Group 내에서의 Distribution과 비교하여 다른 Group간의 분리 정도의 비율로 계산되는 값으로 모든 두 개의 Group 쌍에 대해 각 Group의 크기의 합을 각 Group의 중심 간 거리로 나눈 값으로서 표현되는 함수이다. 즉, 값이 작을수록 최적화된 군집이라고 할 수 있다.



cluster k 개수 : 2
Silhouette Coefficient: 1.0000
Davies Bouldin Index: 0.0000
cluster k 개수 : 3
Silhouette Coefficient: 0.9999
Davies Bouldin Index: 0.0000
cluster k 개수 : 4
Silhouette Coefficient: 0.9948
Davies Bouldin Index: 0.1940
cluster k 개수 : 5
Silhouette Coefficient: 0.9008
Davies Bouldin Index: 0.4172
cluster k 개수 : 6
Silhouette Coefficient: 0.8145
Davies Bouldin Index: 0.5310
cluster k 개수 : 7
Silhouette Coefficient: 0.7984
Davies Bouldin Index: 0.5654
cluster k 개수 : 8
Silhouette Coefficient: 0.8530
Davies Bouldin Index: 0.5008
cluster k 개수 : 9
Silhouette Coefficient: 0.6349
Davies Bouldin Index: 0.5763
cluster k 개수 : 10
Silhouette Coefficient: 0.7120
Davies Bouldin Index: 0.6476
cluster k 개수 : 11
Silhouette Coefficient: 0.6846
Davies Bouldin Index: 0.5915
cluster k 개수 : 12
Silhouette Coefficient: 0.5867
Davies Bouldin Index: 0.5952
cluster k 개수 : 13
Silhouette Coefficient: 0.5508
Davies Bouldin Index: 0.6322
cluster k 개수 : 14
Silhouette Coefficient: 0.3205
Davies Bouldin Index: 0

In [ ]:
from sklearn.cluster import KMeans
import scipy.stats as ss





cluster_df = knn_fill_df[['유동자산', '매출채권', '비유동자산']]

# Z 표준화 - 유클리디안 거리를 기반으로 하는 KMeans이기 때문에 변수간 유사도 비교를 위해 표준화 진행
for i in cluster_df.columns:
  cluster_df[i] = ss.zscore(cluster_df[i])


model = KMeans(n_clusters=10)
model.fit(cluster_df)
knn_fill_df['cluster'] = model.labels_




In [ ]:
# K=4개일때
knn_fill_df['cluster'].value_counts()

0    108398
1       739
3         4
2         1
Name: cluster, dtype: int64

In [ ]:
# K=6개일때
knn_fill_df['cluster'].value_counts()

0    106583
3      1715
1       739
5       100
4         4
2         1
Name: cluster, dtype: int64

In [ ]:
# K=7개일때
knn_fill_df['cluster'].value_counts()

0    103908
5      3065
6      1287
1       739
4       138
3         4
2         1
Name: cluster, dtype: int64

In [ ]:
# K=8개일때
knn_fill_df['cluster'].value_counts()

0    103455
7      2484
6      2321
1       739
4       138
5         3
2         1
3         1
Name: cluster, dtype: int64

In [ ]:
# K=9개일때
knn_fill_df['cluster'].value_counts()

0    93760
6    10976
7     2269
4     1278
1      739
5      115
3        3
2        1
8        1
Name: cluster, dtype: int64

In [ ]:
# K=10개일때
knn_fill_df['cluster'].value_counts()

3    94391
6     9499
0     3532
1      739
8      458
5      451
7       67
4        3
2        1
9        1
Name: cluster, dtype: int64

In [ ]:
# K=11개일때
knn_fill_df['cluster'].value_counts()

In [ ]:
# 한개짜리는 안채워질수 있으므로 drop


In [ ]:
# 클러스터간 평균값으로 채우기 - 컬럼별로 채운다.
def fill_na_by_cluster_mean(df):
  for i in list(df.columns):
    df[i].fillna(df.groupby('cluster')[i].transform('mean'), inplace=True)
  return df

final_fs = fill_na_by_cluster_mean(knn_fill_df)
final_fs



,사업자번호,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,자본금,이익잉여금(결손금）,자본총계,매출액,판매비와관리비,영업이익（손실）,법인세비용차감전순손익,법인세비용,당기순이익(손실),기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,영업비율(%),금융비용대매출액비율(%,금융비용대부채비율(%),금융비용대총비용비율(%,부채비율(%),차입금의존도(%),자기자본비율(%),순운전자본비율(%),유동부채비율(%),비유동부채비율(%),부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),미수금,매출원가,무형자산,재고자산,cluster
0,1018100340,20181231,2667731.0,127320.0,32346444.0,1179096.0,35014174.0,422872.0,16176067.0,16598939.0,721100.0,3859072.0,18415235.0,6629483.0,6210864.0,418619.0,3144847.0,635665.0,2509182.000,6.161,4.877,2.368200e+02,1.000000e+02,6.315000e+00,37.849,6.656200e+01,6.107600e+01,9.368500e+01,1.000000e+12,0.000000e+00,1.000000e+12,90.137,1.000000e+12,52.594,6.411000e+00,2.296000e+00,8.784100e+01,2.503810e+02,1.630000e-01,1.000000e+12,4.406200e+01,1.000000e+12,3.243000e+03,1.703222e+07,2.191455e+06,3.592992e+06,0
1,1018100340,20191231,2168837.0,94085.0,34151633.0,1209369.0,36320470.0,10553878.0,5240445.0,15794323.0,721100.0,3708092.0,20526147.0,6541583.0,5698511.0,843072.0,3446082.0,712661.0,2733421.000,7.600,4.300,2.159000e+02,1.000000e+02,1.290000e+01,41.800,6.260000e+01,-3.171751e+05,8.710000e+01,1.000000e+12,0.000000e+00,1.000000e+12,76.900,0.000000e+00,56.500,-2.310000e+01,5.140000e+01,2.550000e+01,2.414000e+02,1.840000e-01,1.000000e+12,5.909200e+01,1.000000e+12,3.243000e+03,1.703222e+07,2.191455e+06,3.592992e+06,0
2,1018100340,20201231,2272260.0,78612.0,38329680.0,1091153.0,40601940.0,863144.0,15496117.0,16359261.0,721100.0,4230449.0,24242679.0,5933317.0,5865171.0,68146.0,4180173.0,773416.0,3406757.000,8.858,5.121,2.883210e+02,1.000000e+02,1.149000e+00,57.417,5.860200e+01,-1.336705e+06,9.885200e+01,1.000000e+12,0.000000e+00,1.000000e+12,67.481,1.000000e+12,59.708,3.471000e+00,3.560000e+00,6.392100e+01,2.757190e+02,1.540000e-01,1.000000e+12,6.871400e+01,1.000000e+12,3.243000e+03,1.703222e+07,2.191455e+06,3.592992e+06,0
3,1018100340,20211231,1175030.0,273522.0,37135811.0,993025.0,38310842.0,9494272.0,4901383.0,14395654.0,721100.0,5169522.0,23915187.0,6158586.0,5771585.0,387000.0,4731471.0,907999.0,3823473.000,9.690,7.878,4.185490e+02,1.000000e+02,6.284000e+00,62.084,5.522800e+01,-6.344112e+05,9.371600e+01,1.000000e+12,0.000000e+00,1.000000e+12,60.195,1.000000e+12,62.424,-2.171500e+01,3.970000e+01,2.049500e+01,2.337490e+02,1.560000e-01,1.000000e+12,3.497900e+01,1.000000e+12,3.243000e+03,1.703222e+07,2.191455e+06,3.592992e+06,0
0,1018100772,20181231,422146.0,17011.0,20370096.0,20352846.0,20792242.0,2684934.0,11404637.0,14089571.0,52000.0,-7410129.0,6702671.0,1127092.0,446655.0,155690.0,20159.0,20159.0,1.231,-32.209,-12878.763,5.344200e+01,1.381300e+01,1.789000e+00,98.361,9.849600e+01,8.618600e+01,2.059900e+01,1.682000e+00,1.918600e+01,2.102080e+02,38.349,3.223600e+01,-10.883,4.005800e+01,1.701510e+02,1.250082e+03,5.500000e-02,1.000000e+12,6.971100e+01,1.000000e+12,3.868800e+05,5.247460e+05,1.703222e+07,2.191455e+06,3.592992e+06,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,8998800427,20211231,9192992.0,2918329.0,9735232.0,9409374.0,18928223.0,11017981.0,6619380.0,17637362.0,514500.0,-1183183.0,1290862.0,25163483.0,3339379.0,-1546667.0,-2254908.0,85659.0,-2254908.000,-11.505,4.102,1.508960e+02,7.124000e+00,-6.147000e+00,-8.961,1.087830e+02,-5.390522e+06,1.061470e+02,1.172000e+00,2.001000e+00,1.056000e+00,1366.324,5.320700e+01,6.820,-9.642000e+00,8.535370e+02,5.127880e+02,7.009100e+01,1.477000e+00,6.785000e+00,7.950000e+00,9.792000e+00,3.317350e+05,2.337077e+07,7.545100e+04,3.680509e+06,0
0,8998800785,20181231,3608435.0,1201346.0,271541001.0,271541001.0,275149436.0,164335821.0,76480438.0,240816259.0,37780000.0,-3315168.0,34333177.0,1099319.0,2612134.0,-2612134.0,-2592341.0,-2592341.0,-1.690,-1.205,-8.775,1.000000e+12,1.000000e+12,1.000000e+12,12726.477,1.272648e+04,1.000000e+12,1.000000e+12,0.

In [ ]:
final_fs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109142 entries, 0 to 3
Data columns (total 47 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   사업자번호         109142 non-null  int64  
 1   결산년월          109142 non-null  int64  
 2   유동자산          109142 non-null  float64
 3   매출채권          109142 non-null  float64
 4   비유동자산         109142 non-null  float64
 5   유형자산          109142 non-null  float64
 6   자산총계          109142 non-null  float64
 7   유동부채          109142 non-null  float64
 8   비유동부채         109142 non-null  float64
 9   부  채  총  계    109142 non-null  float64
 10  자본금           109142 non-null  float64
 11  이익잉여금(결손금）    109142 non-null  float64
 12  자본총계          109142 non-null  float64
 13  매출액           109142 non-null  float64
 14  판매비와관리비       109142 non-null  float64
 15  영업이익（손실）      109142 non-null  float64
 16  법인세비용차감전순손익   109142 non-null  float64
 17  법인세비용         109142 non-null  float64
 18  당기순이익(손실)

In [ ]:
# 결측치 있는 row 삭제
final_fs.dropna(axis=0,inplace=True)

In [ ]:
final_fs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108401 entries, 0 to 3
Data columns (total 47 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   사업자번호         108401 non-null  int64  
 1   결산년월          108401 non-null  int64  
 2   유동자산          108401 non-null  float64
 3   매출채권          108401 non-null  float64
 4   비유동자산         108401 non-null  float64
 5   유형자산          108401 non-null  float64
 6   자산총계          108401 non-null  float64
 7   유동부채          108401 non-null  float64
 8   비유동부채         108401 non-null  float64
 9   부  채  총  계    108401 non-null  float64
 10  자본금           108401 non-null  float64
 11  이익잉여금(결손금）    108401 non-null  float64
 12  자본총계          108401 non-null  float64
 13  매출액           108401 non-null  float64
 14  판매비와관리비       108401 non-null  float64
 15  영업이익（손실）      108401 non-null  float64
 16  법인세비용차감전순손익   108401 non-null  float64
 17  법인세비용         108401 non-null  float64
 18  당기순이익(손실)

# 기존 데이터에서 feature selection

In [ ]:


def make_preprocessing(active, closing, active_important, closing_important):

  active['휴폐업_여부']=[0]*len(active)
  closing['휴폐업_여부'] = [1]*len(closing)
  df = pd.concat([active, closing], axis=0)

  df['CMP_PFIX_NM'].fillna('', inplace=True)
  df['CMP_SFIX_NM'].fillna('', inplace=True)

  # CMP_PFIX_NM, CMP_NM, CMP_SFIX_NM, CMP_ENM 데이터 가공처리
  # (주)가 있는지 없는지 여부만 판단하는 컬럼으로 합쳐준다.
  # CMP_PFIX_NM, CMP_SFIX_NM은 drop ! 
  # CMP_ENM(기업영문명) drop
  df['tmp'] = df['CMP_PFIX_NM'] + df['CMP_SFIX_NM']
  df['(주)여부'] = df.apply(lambda x: 1 if '(주)' in x['tmp'] else 0, axis=1)

  df.drop(['CMP_PFIX_NM', 'CMP_SFIX_NM', 'CMP_ENM', 'tmp'], axis=1, inplace=True)


  # 공기업구분여부도 모델에 큰영향을 줄 것 같지 않으므로 drop한다. - 액티브든, 휴폐업이든 공기업인 기업이 없다.
  df.drop('PBCO_GB', axis=1,inplace=True)

  # 개인법인구분에서 모드 법인으로 나오므로 이것도 컬럼 삭제한다. - 분류모델에 유의미한 영향을 주지 못한다.
  df.drop('PSN_CORP_GB', axis=1, inplace=True)

  
  # 본점지점구분은 그래도 좀 나오니깐 일단 컬럼으로 넣어보고 성능에 유의미한 영향을 주지않으면 빼자
  # 국외투자법인여부는 나름 유의미하게 분포되어 있는 것 같다. 효과는 미비할 것 같지만 그나마 쓸만할 것 같아서 빼지않는다.
  # 벤처기업여부 또한 나름 유의미하게 분포.

  # 상장코드는 있는게 매우 드물다.
  # 상장코드 - 있으면 1, 없으면 0으로 채워넣자
  df['LIST_CD'].fillna(0, inplace=True)
  df['LIST_CD'] = df['LIST_CD'].apply(lambda x: 1 if x!=0 else 0)


  # 산업코드차수 전체에서 10개 빼고 모두 '10'으로 라벨링 돼있다. 그냥 drop
  df.drop('IND_CD_ORDR', axis=1, inplace = True)


  # 산업코드 보류하자 이거 진짜 어렵다...............
  # 산업코드1, 2, 3
  # 일단 null 값은 etc로 통합 - 12개정도밖에 없다.
  # 산업코드 1에서 10개 이하로 나오는 산업코드는 etc로 통합해준다. -> 나중에 분류 모델링의 성능평가를 위해
  # 아니면 이것도 count값을 넣어줄까
  df['IND_CD1'].fillna('etc', inplace=True)
  change_index= []
  for i in range(len(df['IND_CD1'].value_counts())):
    if df['IND_CD1'].value_counts().values[i]<=10:
      change_index.append(df['IND_CD1'].value_counts().index[i])
  df['IND_CD1'].replace(change_index, 'etc', inplace=True)


  # 산업코드 2, 3은 널값이 많으니까 drop
  df.drop(['IND_CD2','IND_CD3'], axis=1, inplace=True)

  # 공공기관 유형
  # 값의 차이가 너무 미세하다.
  # drop
  df.drop('PB_ORG_TYP', axis=1, inplace=True)


  # 중견기업보호여부 일단 넣자. 막 유의미해보이진 않는데 그래도
  df['MDSCO_PRTC_YN'].fillna('N', inplace=True)


  # 본점기업코드
  # 널값은 etc로 채워주고
  # 본점기업코드는 value_counts의 개수를 대신 넣어주면 좋을 것같다. 
  # 숫자가 높을수록 체인점을 많이 보유한 업종임을 판단할 수 있다.
  # 시간이 너무 오래걸린다.
  df['HDOF_CMP_CD'].fillna('etc', inplace=True)
  tmp =[] # 개수가 얼마나 나왔는지 저장을 위해
  for i in df['HDOF_CMP_CD']:
    tmp.append(list(df['HDOF_CMP_CD']).count(i))
  df['HDOF_CMP_CD'] = tmp



  # 설립일자 널값은 어떻게 채울까, 일단 최빈값으로 채우자
  # 중요한 데이터면 인터넷 검색해서 채우면 된다.
  # df['ESTB_DATE'].fillna(df['ESTB_DATE'].mode(), inplace = True)

  # 설립구분은 널값 없다. 일단 넣는다.
  df['ESTB_GB'] = df['ESTB_GB'].astype('int')

  # 상장일자는 널값인게 매우 많다.
  # 널값이면 0, 아니면 1로 채운다. - 상장했는지 안했는지 여부만 확인하려고 ?
  df['LIST_DATE'].fillna(0, inplace=True)
  df['LIST_DATE'] = df['LIST_DATE'].apply(lambda x: 1 if x!=0 else 0)

  # 주요사업내용
  df['MN_BIZ_CONT'].fillna('etc', inplace=True)

  # 국가명
  # 나중에 gdp나 중소기업 지원정책이나 이런거 파생변수로 넣을 수 있을듯
  df['NATN_NM'] = df['NATN_NM'].fillna('etc')



  # 홈페이지 url
  # 홈페이지 url에 있는지 없는지 여부에 따라 1, 0으로 라벨링
  df['HOMEPAGE_URL'].fillna(0, inplace=True)
  df['HOMEPAGE_URL'] =  df['HOMEPAGE_URL'].apply(lambda x:0 if x==0 else 1)


  # 대표자명
  df['CEO_NM'].fillna('etc', inplace=True)



  # 직원수
  # 널값은 일단 0으로 채운다.
  df['EMP_CNT'].fillna(0, inplace=True)
  


  ########################################################### 설립일자 보류 #############################################
  # 기간을 계산할수있으면 좋을듯 today랑 빼든 종료일자랑 빼든 시작일자랑 빼든
  # 설립일자 널값 어떻게 채워줄까

  ########################################################### 주요사업내용 보류 #########################################
  # 자연어 처리로 활용할 방안 있는지 고려

  ########################################################### 대표자명 보류 #############################################
  # 이것도 대표자명의 건강상태를 통해 파악할만한 것이 있는지 확인




  # active_important, closing_important 전처리 필요
  # 하나의 unique_BIZ_NO 
  # 만약에 하나도 없으면 0, 개수만큼 쓴다. -> 휴폐업구분이 여러개 나오는 것도 있다.
  # for i in active_important['BIZ_NO'].unique():
    


  # for i in closing_important['BIZ_NO'].unique():




  df.reset_index(drop=True, inplace=True)







  return df

In [ ]:
df = make_preprocessing(active, closing, active_important, closing_important)

In [ ]:
df.head(10)

,BIZ_NO,CMP_NM,BZ_TYP,CMP_SCL,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,휴폐업_여부,(주)여부
0,1138111739,한일가스산업,M,2,1,N,N,0,47712.0,N,31363,19660419.0,1,0,LP Gas 충전 및 판매업,etc,1,홍순철외 1명,43.0,0,1
1,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1
2,1318167565,도영운수,M,2,1,N,N,0,49212.0,N,31363,20011030.0,1,0,시내버스업,etc,1,김명화,149.0,0,1
3,5048128251,중원산업,M,2,1,N,N,0,30391.0,N,31363,19990112.0,1,0,자동차 부품 제조,etc,1,김명구,75.0,0,1
4,1098164260,카타나골프,M,2,1,N,N,0,46800.0,N,31363,20000531.0,1,0,골프용품 전반의 제조 및 수출업,etc,0,김홍득,3.0,0,1
5,2148612093,센트로,M,2,1,N,N,0,55101.0,N,31363,19880810.0,1,0,호텔운영 및 객장임대업,etc,1,이연신,2.0,0,1
6,2068148256,엄앤드이종합건축사사무소,M,2,1,N,N,0,72110.0,N,31363,19940120.0,1,0,"설계,감리",etc,1,이각표외 3명,200.0,0,1
7,1078156099,세풍폴리머,M,2,1,N,N,0,46733.0,N,31363,19960701.0,1,0,플라스틱 합성수지 제조업체,etc,1,김종만,36.0,0,1
8,6218111642,제일산업,M,2,1,N,N,0,etc,N,31363,19931021.0,1,0,마대원단 직조 및 판매업,etc,1,정득부,17.0,0,1
9,5148140835,한국대풍,M,2,1,Y,N,0,30310.0,N,31363,20010621.0,1,0,"자동차부품의 제조,판매,수출입",일본,1,아와츠시게키외 1명,94.0,0,1


In [ ]:
# 휴폐업 이력 있는 액티브 데이터 삭제
active_del_list = list(active_important['BIZ_NO'].unique())

del_idx = []
for i in range(len(df)):
  if df['BIZ_NO'][i] in active_del_list:
    del_idx.append(i)

df.drop(del_idx,axis=0, inplace = True)
df = df.reset_index(drop=True)

In [ ]:
final_df = pd.merge(df, final_fs, left_on = 'BIZ_NO', right_on = '사업자번호', how='left')

In [ ]:
final_df

,BIZ_NO,CMP_NM,BZ_TYP,CMP_SCL,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,휴폐업_여부,(주)여부,사업자번호,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,자본금,이익잉여금(결손금）,자본총계,매출액,판매비와관리비,영업이익（손실）,법인세비용차감전순손익,법인세비용,당기순이익(손실),기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,영업비율(%),금융비용대매출액비율(%,금융비용대부채비율(%),금융비용대총비용비율(%,부채비율(%),차입금의존도(%),자기자본비율(%),순운전자본비율(%),유동부채비율(%),비유동부채비율(%),부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),미수금,매출원가,무형자산,재고자산,cluster
0,1138111739,한일가스산업,M,2,1,N,N,0,47712.0,N,31363,19660419.0,1,0,LP Gas 충전 및 판매업,etc,1,홍순철외 1명,43.0,0,1,1.138112e+09,20211231.0,2758846.0,1577971.0,11235042.0,5705994.0,13993889.0,6172590.0,6343124.0,12515714.0,300000.0,1178175.0,1478175.0,19738142.0,5176975.0,-979131.0,-812712.0,-8.127120e+05,1.000000e+12,8.419,392.725,2.126800e+01,-4.961000e+00,-4.118000e+00,104.027,-2099711.928,1.049610e+02,3.230000e-01,1.000000e+12,0.303,8.467000e+02,2.416500e+01,1.056300e+01,-2.439500e+01,4.175820e+02,4.291190e+02,6.340900e+01,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,100333.000,15540298.0,2.500000e+04,5.917000e+04,3.592992e+06,0.0
1,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20191231.0,4772148.0,1203628.0,6784222.0,4378120.0,11556370.0,3769840.0,3756536.0,7526376.0,100000.0,3929994.0,4029994.0,18826244.0,1051725.0,1327150.0,1017636.0,2.053280e+05,8.123080e+05,9.200,34.000,3.930000e+03,1.260000e+01,7.000000e+00,4.300,94.600,4.999993e+11,9.300000e+01,9.000000e-01,2.400,1.000000e+00,1.868000e+02,2.490000e+01,3.490000e+01,8.700000e+00,9.350000e+01,9.320000e+01,4.000000e+01,1.746000e+00,1.039200e+01,3.128200e+01,47.134,433886.0,1.644737e+07,2.080000e+02,1.792159e+06,0.0
2,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20201231.0,6096808.0,1229902.0,7819289.0,4307102.0,13916097.0,5397205.0,4348030.0,9745234.0,100000.0,4070863.0,4170863.0,13852434.0,900743.0,-46273.0,170235.0,2.936600e+04,1.408690e+05,2.355,29.253,4.070863e+03,6.168000e+00,-3.340000e-01,1.017,98.807,1.000000e+12,1.003340e+02,1.149000e+00,1.842,1.129000e+00,2.336500e+02,4.015500e+01,2.997200e+01,5.027000e+00,1.294030e+02,1.042480e+02,7.035000e+01,1.088000e+00,6.019000e+00,1.138500e+01,12.691,445094.0,1.299796e+07,2.080000e+02,2.810573e+06,0.0
3,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2.078132e+09,20211231.0,5562280.0,1443248.0,8236579.0,4351158.0,13798860.0,5247799.0,4002659.0,9250457.0,100000.0,4448402.0,4548402.0,14889722.0,765533.0,452145.0,391144.0,1.360500e+04,3.775390e+05,3.881,32.238,4.448402e+03,8.178000e+00,3.037000e+00,2.536,97.398,-1.352845e+06,9.696300e+01,1.076000e+00,1.687,1.094000e+00,2.033780e+02,4.045100e+01,3.296200e+01,2.279000e+00,1.153770e+02,8.800100e+01,6.212700e+01,1.074000e+00,5.800000e+00,1.114000e+01,12.710,429334.0,1.367204e+07,2.080000e+02,2.324076e+06,0.0
4,1318167565,도영운수,M,2,1,N,N,0,49212.0,N,31363,20011030.0,1,0,시내버스업,etc,1,김명화,149.0,0,1,1.318168e+09,20191231.0,1241591.0,990206.0,3341364.0,3195475.0,4582955.0,2349133.0,1153634.5,2349133.0,200000.0,2033822.0,2233822.0,11341536.0,846751.0,-67323.0,-183472.0,3.779400e+04,-2.212650e+05,-4.640,44.378,1.016911e+03,6.872000e+00,-5.940000e-01,-1.951,101.605,1.000000e+12,1.005940e+02,2.000000e-03,0.010,2.000000e-03,1.051620e+02,1.434700e+01,4.874200e+01,-2.416700e+01,1.051620e+02,1.000000e+12,2.071300e+01,2.381000e+00,1.000000e+12,2.290700e+01,93.781,165.0,1.056211e+07,1.797000e+03,3.592992e+06,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107482,8018501578,우진푸드중부지점,M,2,2,N,N,0,etc,N,1,20200727.0,1,0,"돈

In [ ]:
# 모든 재무제표가 null인거 삭제

print(len(final_df[final_df['결산년월'].isnull()])) # 273개 나온다

# 273개 row 삭제
final_df.drop(list(final_df[final_df['결산년월'].isnull()].index) ,axis=0, inplace=True)
final_df = final_df.reset_index(drop=True)

273


In [ ]:
# aa = final_df[final_df['BIZ_NO']==2078132193].iloc[-1]
# pd.DataFrame(aa).T

,BIZ_NO,CMP_NM,BZ_TYP,CMP_SCL,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,휴폐업_여부,(주)여부,사업자번호,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,자본금,이익잉여금(결손금）,자본총계,매출액,판매비와관리비,영업이익（손실）,법인세비용차감전순손익,법인세비용,당기순이익(손실),기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,영업비율(%),금융비용대매출액비율(%,금융비용대부채비율(%),금융비용대총비용비율(%,부채비율(%),차입금의존도(%),자기자본비율(%),순운전자본비율(%),유동부채비율(%),비유동부채비율(%),부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),미수금,매출원가,무형자산,재고자산,cluster
3,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2078132193.0,20211231.0,5562280.0,1443248.0,8236579.0,4351158.0,13798860.0,5247799.0,4002659.0,9250457.0,100000.0,4448402.0,4548402.0,14889722.0,765533.0,452145.0,391144.0,13605.0,377539.0,3.881,32.238,4448.402,8.178,3.037,2.536,97.398,-1352845.013,96.963,1.076,1.687,1.094,203.378,40.451,32.962,2.279,115.377,88.001,62.127,1.074,5.8,11.14,12.71,429334.0,13672044.0,208.0,2324076.0,0.0


In [ ]:
# 결산년월 맨마지막 행만 들고오기
real_final_df = pd.DataFrame()
for i in list(final_df['BIZ_NO'].unique()):
  aa= final_df[final_df['BIZ_NO']==i].iloc[-1]
  tmp = pd.DataFrame(aa).T
  real_final_df = pd.concat([real_final_df, tmp],axis=0)
real_final_df.reset_index(drop=True)

,BIZ_NO,CMP_NM,BZ_TYP,CMP_SCL,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,휴폐업_여부,(주)여부,사업자번호,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,자본금,이익잉여금(결손금）,자본총계,매출액,판매비와관리비,영업이익（손실）,법인세비용차감전순손익,법인세비용,당기순이익(손실),기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,영업비율(%),금융비용대매출액비율(%,금융비용대부채비율(%),금융비용대총비용비율(%,부채비율(%),차입금의존도(%),자기자본비율(%),순운전자본비율(%),유동부채비율(%),비유동부채비율(%),부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),미수금,매출원가,무형자산,재고자산,cluster
0,1138111739,한일가스산업,M,2,1,N,N,0,47712.0,N,31363,19660419.0,1,0,LP Gas 충전 및 판매업,etc,1,홍순철외 1명,43.0,0,1,1138111739.0,20211231.0,2758846.0,1577971.0,11235042.0,5705994.0,13993889.0,6172590.0,6343124.0,12515714.0,300000.0,1178175.0,1478175.0,19738142.0,5176975.0,-979131.0,-812712.0,-812712.0,1000000000000.0,8.419,392.725,21.268,-4.961,-4.118,104.027,-2099711.928,104.961,0.323,1000000000000.0,0.303,846.7,24.165,10.563,-24.395,417.582,429.119,63.409,1000000000000.0,1000000000000.0,1000000000000.0,1000000000000.0,100333.0,15540298.0,25000.0,59170.0,3592991.625806,0.0
1,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2078132193.0,20211231.0,5562280.0,1443248.0,8236579.0,4351158.0,13798860.0,5247799.0,4002659.0,9250457.0,100000.0,4448402.0,4548402.0,14889722.0,765533.0,452145.0,391144.0,13605.0,377539.0,3.881,32.238,4448.402,8.178,3.037,2.536,97.398,-1352845.013,96.963,1.076,1.687,1.094,203.378,40.451,32.962,2.279,115.377,88.001,62.127,1.074,5.8,11.14,12.71,429334.0,13672044.0,208.0,2324076.0,0.0
2,1318167565,도영운수,M,2,1,N,N,0,49212.0,N,31363,20011030.0,1,0,시내버스업,etc,1,김명화,149.0,0,1,1318167565.0,20211231.0,1965055.0,1389692.0,4358581.0,4232380.0,6323635.0,3749079.0,1024357.0,4773436.0,200000.0,1350200.0,1550200.0,12582033.0,897080.0,-20771.0,-482373.0,37794.0,-482373.0,-6.641,21.352,675.1,6.965,-0.165,-3.834,103.824,1595.847,100.165,0.383,1.016,0.368,307.924,23.993,24.514,-28.212,241.845,66.079,37.939,1.925,1000000000000.0,9.85,67.045,520.0,11705724.0,917.0,3592991.625806,0.0
3,5048128251,중원산업,M,2,1,N,N,0,30391.0,N,31363,19990112.0,1,0,자동차 부품 제조,etc,1,김명구,75.0,0,1,5048128251.0,20211231.0,12668826.0,5749707.0,15954138.0,13741069.0,28622964.0,9393776.0,3998392.0,13392168.0,50000.0,15180796.0,15230796.0,35437416.0,3491275.0,2124877.0,3044086.0,618646.0,2425439.0,9.158,49.543,28361.592,15.848,5.996,6.844,91.682,-4780308.293,94.004,0.351,0.924,0.371,87.928,18.874,53.212,11.442,61.676,26.252,37.791,1.273,49.201,6.387,7.718,11688.0,29821265.0,474611.0,1035680.0,0.0
4,1098164260,카타나골프,M,2,1,N,N,0,46800.0,N,31363,20000531.0,1,0,골프용품 전반의 제조 및 수출업,etc,0,김홍득,3.0,0,1,1098164260.0,20211231.0,10804850.0,137054.0,1146992.0,980971.0,11951841.0,6960351.0,4780000.0,11740351.0,100000.0,111490.0,211490.0,1172000.0,479947.0,12032.0,-146752.0,44086.0,-190838.0,-0.139,0.933,111.49,41.978,1.027,-16.283,112.266,1000000000000.0,98.973,14.747,1.364,12.867,5551.256,73.186,1.77,32.167,3291.102,2260.154,1001.736,0.09,1.44,3.294,4.184,24272.0,680021.0,815935.0,3592991.625806,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35081,4808501629,게방식당팩토리,M,2,2,N,N,0,etc,N,1,20180126.0,1,0,간장게장(장류),etc,1,방건혁,0.0,1,0,4808501629.0,20201231.0,1482364.0,213652.0,585666.0,271590.0,2068030.0,899545.0,780000.0,1679545.0,264910.0,123575.0,388485.0,2937061.0,1327271.0,99950.0,92983.0,3083.0,89900.0,5.708,5.976,46.648,48.594,3.403,3.061,96.84,3850.485,96.597,0.293,0.623,0.302,432.332,57.076,18.785,28.182,231.552,200.78,57.185,1.702,3.775,27.392,17.301,164.0,1509840.0,177988.0,927919.0,0.0
35082,8018501578,우진푸드중부지점,M,2,2,N,N,0,etc,N,1,20200727.0,1,0,"돈가스, 탕수육, 꿔바로우",etc,1,신남정,22.0,1,1,8018501578.0,20201231.0,1206945.0,731340.0,639

In [ ]:
real_final_df

,BIZ_NO,CMP_NM,BZ_TYP,CMP_SCL,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_DATE,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,휴폐업_여부,(주)여부,사업자번호,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,자본금,이익잉여금(결손금）,자본총계,매출액,판매비와관리비,영업이익（손실）,법인세비용차감전순손익,법인세비용,당기순이익(손실),기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,영업비율(%),금융비용대매출액비율(%,금융비용대부채비율(%),금융비용대총비용비율(%,부채비율(%),차입금의존도(%),자기자본비율(%),순운전자본비율(%),유동부채비율(%),비유동부채비율(%),부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),미수금,매출원가,무형자산,재고자산,cluster
0,1138111739,한일가스산업,M,2,1,N,N,0,47712.0,N,31363,19660419.0,1,0,LP Gas 충전 및 판매업,etc,1,홍순철외 1명,43.0,0,1,1138111739.0,20211231.0,2758846.0,1577971.0,11235042.0,5705994.0,13993889.0,6172590.0,6343124.0,12515714.0,300000.0,1178175.0,1478175.0,19738142.0,5176975.0,-979131.0,-812712.0,-812712.0,1000000000000.0,8.419,392.725,21.268,-4.961,-4.118,104.027,-2099711.928,104.961,0.323,1000000000000.0,0.303,846.7,24.165,10.563,-24.395,417.582,429.119,63.409,1000000000000.0,1000000000000.0,1000000000000.0,1000000000000.0,100333.0,15540298.0,25000.0,59170.0,3592991.625806,0.0
3,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,19910806.0,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,2078132193.0,20211231.0,5562280.0,1443248.0,8236579.0,4351158.0,13798860.0,5247799.0,4002659.0,9250457.0,100000.0,4448402.0,4548402.0,14889722.0,765533.0,452145.0,391144.0,13605.0,377539.0,3.881,32.238,4448.402,8.178,3.037,2.536,97.398,-1352845.013,96.963,1.076,1.687,1.094,203.378,40.451,32.962,2.279,115.377,88.001,62.127,1.074,5.8,11.14,12.71,429334.0,13672044.0,208.0,2324076.0,0.0
6,1318167565,도영운수,M,2,1,N,N,0,49212.0,N,31363,20011030.0,1,0,시내버스업,etc,1,김명화,149.0,0,1,1318167565.0,20211231.0,1965055.0,1389692.0,4358581.0,4232380.0,6323635.0,3749079.0,1024357.0,4773436.0,200000.0,1350200.0,1550200.0,12582033.0,897080.0,-20771.0,-482373.0,37794.0,-482373.0,-6.641,21.352,675.1,6.965,-0.165,-3.834,103.824,1595.847,100.165,0.383,1.016,0.368,307.924,23.993,24.514,-28.212,241.845,66.079,37.939,1.925,1000000000000.0,9.85,67.045,520.0,11705724.0,917.0,3592991.625806,0.0
10,5048128251,중원산업,M,2,1,N,N,0,30391.0,N,31363,19990112.0,1,0,자동차 부품 제조,etc,1,김명구,75.0,0,1,5048128251.0,20211231.0,12668826.0,5749707.0,15954138.0,13741069.0,28622964.0,9393776.0,3998392.0,13392168.0,50000.0,15180796.0,15230796.0,35437416.0,3491275.0,2124877.0,3044086.0,618646.0,2425439.0,9.158,49.543,28361.592,15.848,5.996,6.844,91.682,-4780308.293,94.004,0.351,0.924,0.371,87.928,18.874,53.212,11.442,61.676,26.252,37.791,1.273,49.201,6.387,7.718,11688.0,29821265.0,474611.0,1035680.0,0.0
12,1098164260,카타나골프,M,2,1,N,N,0,46800.0,N,31363,20000531.0,1,0,골프용품 전반의 제조 및 수출업,etc,0,김홍득,3.0,0,1,1098164260.0,20211231.0,10804850.0,137054.0,1146992.0,980971.0,11951841.0,6960351.0,4780000.0,11740351.0,100000.0,111490.0,211490.0,1172000.0,479947.0,12032.0,-146752.0,44086.0,-190838.0,-0.139,0.933,111.49,41.978,1.027,-16.283,112.266,1000000000000.0,98.973,14.747,1.364,12.867,5551.256,73.186,1.77,32.167,3291.102,2260.154,1001.736,0.09,1.44,3.294,4.184,24272.0,680021.0,815935.0,3592991.625806,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107206,4808501629,게방식당팩토리,M,2,2,N,N,0,etc,N,1,20180126.0,1,0,간장게장(장류),etc,1,방건혁,0.0,1,0,4808501629.0,20201231.0,1482364.0,213652.0,585666.0,271590.0,2068030.0,899545.0,780000.0,1679545.0,264910.0,123575.0,388485.0,2937061.0,1327271.0,99950.0,92983.0,3083.0,89900.0,5.708,5.976,46.648,48.594,3.403,3.061,96.84,3850.485,96.597,0.293,0.623,0.302,432.332,57.076,18.785,28.182,231.552,200.78,57.185,1.702,3.775,27.392,17.301,164.0,1509840.0,177988.0,927919.0,0.0
107209,8018501578,우진푸드중부지점,M,2,2,N,N,0,etc,N,1,20200727.0,1,0,"돈가스, 탕수육, 꿔바로우",etc,1,신남정,22.0,1,1,8018501578.0,20201231.0,1206945.0,731340.0

In [ ]:
# 필요없는 feature drop
# 설립일자, 사업자번호, 클러스터
real_final_df.drop(['ESTB_DATE', '사업자번호', 'cluster'],axis=1, inplace =True)
real_final_df.isna().sum().sum()

0

In [ ]:
real_final_df.to_csv('real_baseline_df.csv', index = False)

In [ ]:
real_final_df['휴폐업_여부'].value_counts()

0    28382
1     6704
Name: 휴폐업_여부, dtype: int64

# Feature 추가하기


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/real_baseline_df.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35086 entries, 0 to 35085
Data columns (total 65 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   BIZ_NO           35086 non-null  int64  
 1   CMP_NM           35086 non-null  object 
 2   BZ_TYP           35086 non-null  object 
 3   CMP_SCL          35086 non-null  int64  
 4   HDOF_BR_GB       35086 non-null  int64  
 5   FR_IVST_CORP_YN  35086 non-null  object 
 6   VENT_YN          35086 non-null  object 
 7   LIST_CD          35086 non-null  int64  
 8   IND_CD1          35086 non-null  object 
 9   MDSCO_PRTC_YN    35086 non-null  object 
 10  HDOF_CMP_CD      35086 non-null  int64  
 11  ESTB_GB          35086 non-null  int64  
 12  LIST_DATE        35086 non-null  int64  
 13  MN_BIZ_CONT      35086 non-null  object 
 14  NATN_NM          35086 non-null  object 
 15  HOMEPAGE_URL     35086 non-null  int64  
 16  CEO_NM           35086 non-null  object 
 17  EMP_CNT     

### altman k- score

In [ ]:
# 알트만 K score
def altman_k_score(df):
  df['X1'] = np.log(df['자산총계'])
  df['X2'] = np.log(df['총자본회전율(회)'])
  df['X3'] = df['이익잉여금(결손금）'] / df['자산총계']
  df['X4'] = df['자본총계'] / df['자산총계']

  df['K_score'] = -17.862 + 1.472 * df['X1'] + 3.041 * df['X2'] + 14.839 * df['X3'] + 1.516 * df['X4']

  df.drop(['X1','X2', 'X3', 'X4'], axis=1,inplace=True)

  # 부실 라벨링
  # 0.75 < K: 부실가능성 X
  # -2 < K < 0.75 : 판정 유보
  # K< -2 : 부실가능성 심각

  df['K_부실여부판단'] = df.apply(lambda x: '안전기업' if x['K_score']>0.75 else '판정유보기업' if x['K_score'] > -2 else '부실 가능성 높은 기업', axis=1)

  return df


df = altman_k_score(df)
df

,BIZ_NO,CMP_NM,BZ_TYP,CMP_SCL,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,휴폐업_여부,(주)여부,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,자본금,이익잉여금(결손금）,자본총계,매출액,판매비와관리비,영업이익（손실）,법인세비용차감전순손익,법인세비용,당기순이익(손실),기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,영업비율(%),금융비용대매출액비율(%,금융비용대부채비율(%),금융비용대총비용비율(%,부채비율(%),차입금의존도(%),자기자본비율(%),순운전자본비율(%),유동부채비율(%),비유동부채비율(%),부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),미수금,매출원가,무형자산,재고자산,K_score,K_부실여부판단
0,1138111739,한일가스산업,M,2,1,N,N,0,47712.0,N,31363,1,0,LP Gas 충전 및 판매업,etc,1,홍순철외 1명,43.0,0,1,20211231.0,2758846.0,1577971.0,11235042.0,5705994.0,13993889.0,6172590.0,6343124.0,12515714.0,300000.0,1178175.0,1478175.0,19738142.0,5176975.0,-979131.0,-812712.0,-812712.000,1.000000e+12,8.419,392.725,2.126800e+01,-4.961000e+00,-4.118000e+00,104.027,-2099711.928,1.049610e+02,3.230000e-01,1.000000e+12,0.303,8.467000e+02,2.416500e+01,1.056300e+01,-2.439500e+01,4.175820e+02,4.291190e+02,6.340900e+01,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,100333.000,15540298.0,2.500000e+04,5.917000e+04,3.012498e+06,91.793878,안전기업
1,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,20211231.0,5562280.0,1443248.0,8236579.0,4351158.0,13798860.0,5247799.0,4002659.0,9250457.0,100000.0,4448402.0,4548402.0,14889722.0,765533.0,452145.0,391144.0,13605.000,3.775390e+05,3.881,32.238,4.448402e+03,8.178000e+00,3.037000e+00,2.536,97.398,-1.352845e+06,9.696300e+01,1.076000e+00,1.687,1.094000e+00,2.033780e+02,4.045100e+01,3.296200e+01,2.279000e+00,1.153770e+02,8.800100e+01,6.212700e+01,1.074000e+00,5.800000e+00,1.114000e+01,12.710,429334.0,1.367204e+07,2.080000e+02,2.324076e+06,11.838342,안전기업
2,1318167565,도영운수,M,2,1,N,N,0,49212.0,N,31363,1,0,시내버스업,etc,1,김명화,149.0,0,1,20211231.0,1965055.0,1389692.0,4358581.0,4232380.0,6323635.0,3749079.0,1024357.0,4773436.0,200000.0,1350200.0,1550200.0,12582033.0,897080.0,-20771.0,-482373.0,37794.000,-4.823730e+05,-6.641,21.352,6.751000e+02,6.965000e+00,-1.650000e-01,-3.834,103.824,1.595847e+03,1.001650e+02,3.830000e-01,1.016,3.680000e-01,3.079240e+02,2.399300e+01,2.451400e+01,-2.821200e+01,2.418450e+02,6.607900e+01,3.793900e+01,1.925000e+00,1.000000e+12,9.850000e+00,67.045,520.0,1.170572e+07,9.170000e+02,3.012498e+06,10.720871,안전기업
3,5048128251,중원산업,M,2,1,N,N,0,30391.0,N,31363,1,0,자동차 부품 제조,etc,1,김명구,75.0,0,1,20211231.0,12668826.0,5749707.0,15954138.0,13741069.0,28622964.0,9393776.0,3998392.0,13392168.0,50000.0,15180796.0,15230796.0,35437416.0,3491275.0,2124877.0,3044086.0,618646.000,2.425439e+06,9.158,49.543,2.836159e+04,1.584800e+01,5.996000e+00,6.844,91.682,-4.780308e+06,9.400400e+01,3.510000e-01,0.924,3.710000e-01,8.792800e+01,1.887400e+01,5.321200e+01,1.144200e+01,6.167600e+01,2.625200e+01,3.779100e+01,1.273000e+00,4.920100e+01,6.387000e+00,7.718,11688.0,2.982126e+07,4.746110e+05,1.035680e+06,16.822722,안전기업
4,1098164260,카타나골프,M,2,1,N,N,0,46800.0,N,31363,1,0,골프용품 전반의 제조 및 수출업,etc,0,김홍득,3.0,0,1,20211231.0,10804850.0,137054.0,1146992.0,980971.0,11951841.0,6960351.0,4780000.0,11740351.0,100000.0,111490.0,211490.0,1172000.0,479947.0,12032.0,-146752.0,44086.000,-1.908380e+05,-0.139,0.933,1.114900e+02,4.197800e+01,1.027000e+00,-16.283,112.266,1.000000e+12,9.897300e+01,1.474700e+01,1.364,1.286700e+01,5.551256e+03,7.318600e+01,1.770000e+00,3.216700e+01,3.291102e+03,2.260154e+03,1.001736e+03,9.000000e-02,1.440000e+00,3.294000e+00,4.184,24272.0,6.800210e+05,8.159350e+05,3.012498e+06,-1.031020,판정유보기업
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35081,4808501629,게방식당팩토리,M,2,2,N,N,0,etc,N,1,1,0,간장게장(장류),etc,1,방건혁,0.0,1,0,20201231.0,1482364.0,213652.0,585666.0,271590.0,2068030.0,899545.0,780000.0,167954

In [ ]:
df['K_score'].isna().sum()
# 0으로 나눈게 있어서 그런가 널값이 나온다. -> 5개 drop

df.dropna(axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df['K_부실여부판단'].isna().sum()
# 널값 나온것들이 모두 부실 가능성 높은 기업으로 분류됐다.

0

In [ ]:
df['K_부실여부판단'].value_counts()

안전기업            32346
부실 가능성 높은 기업     1937
판정유보기업            798
Name: K_부실여부판단, dtype: int64

In [ ]:
# 최소 일자
print(final_df['결산년월'].min())

# 최대 일자 
print(final_df['결산년월'].max())

20180131.0
20220630.0


## 환율 crawling

In [ ]:
!pip install finance-datareader
def add_환율(df):
  

  df['결산년월'] = df['결산년월'].astype('int')
  df['결산년월'] = df['결산년월'].astype('str')

  import FinanceDataReader as fdr

  # 원달라 환율
  USD_KRX = fdr.DataReader('USD/KRW', start = '2018-01-01') # 미국달러 / 한국 원(원달러)

  # index인 date 컬럼으로 올리기
  USD_KRX = USD_KRX.reset_index()
  USD_KRX = USD_KRX[['Date', 'Close']]

  # 연월별로 평균내기(띄엄띄엄 있다.)
  USD_KRX['Date'] = USD_KRX['Date'].astype('str')
  USD_KRX['연월'] = USD_KRX.apply(lambda x: x['Date'][:7], axis=1)

  date = list(USD_KRX.groupby('연월')['Close'].mean().index)
  close = list(USD_KRX.groupby('연월')['Close'].mean().values)

  환율 = pd.DataFrame()
  환율['연월'] = date
  환율['환율'] = close # 종가를 기준으로

  df['연월'] = df['결산년월'].astype('str')
  df['연월'] = df.apply(lambda x:x['연월'][:4]+'-'+x['연월'][4:6], axis=1)

  df = pd.merge(df, 환율,how = 'left', on='연월')
  
  return df

df = add_환율(df)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 금리지표 크롤링
- 91일 CD금리
- 콜금리
- 국고채
- 회사채
- 참고링크 : https://blog.naver.com/PostView.nhn?blogId=koko8624&logNo=221292348073&parentCategoryNo=&categoryNo=18&viewDate=&isShowPopularPosts=true&from=search

In [ ]:
import requests as re
from bs4 import BeautifulSoup
import pandas as pd
from pandas import DataFrame, Series
import requests as re
from bs4 import BeautifulSoup

url = 'http://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=IRR_CD91&page=1'

crawling_list = ['IRR_CALL',]

#폴더 위치를 입력해주세요
# folder_adress = 'C:/Users/kk/......'

def crawling_interest_rates():
    data_dict = {'IRR_CD91':[],
                 'IRR_CALL':[],
                 'IRR_GOVT03Y':[],
                 'IRR_CORP03Y':[]}

    label_list = ['IRR_CD91','IRR_CALL','IRR_GOVT03Y','IRR_CORP03Y']

    Data = DataFrame()
    
    for label in label_list:
    
        date_list = []
        
        try:
            for i in range(1,171): # 171쪽에 2018년이 시작되므로.
                url = re.get('http://finance.naver.com/marketindex/interestDailyQuote.nhn?marketindexCd=%s&page=%s'%(label,i))
                url = url.content
        
                soup = BeautifulSoup(url,'html.parser')
        
                # 날짜 가져오기
                dates = soup.select('tr > td.date')
            
            
                # 빈페이지인지 테스트
                try:
                    test = soup.find('tbody').find('tr').find('td',{'class':'num'}).text # .text가 에러를 반환하는가?
                except:
                    break
            
            
                # 처음 한번만 가져오자
                for date in dates:
                    date_list.append(date.text.strip())
                    
                                        
        
                rates = soup.find('tbody').find_all('tr')
        
                for rate in rates:
                    data_dict[label].append(rate.find('td',{'class':'num'}).text.strip())
            
        except:
            print('Error')
        
        temp_dataframe = DataFrame(data_dict[label], index = date_list)
        Data = pd.merge(Data,temp_dataframe,how = 'outer', left_index = True, right_index = True)
        
        print(label + '의 자료를 성공적으로 가져왔습니다')
    
    Data.columns = ['CD91일', '콜 금리', '국고채 3년', '회사채 3년']
    # Data.to_csv('%s/interest_rate.csv')
    return Data

금리 = crawling_interest_rates()






IRR_CD91의 자료를 성공적으로 가져왔습니다
IRR_CALL의 자료를 성공적으로 가져왔습니다
IRR_GOVT03Y의 자료를 성공적으로 가져왔습니다
IRR_CORP03Y의 자료를 성공적으로 가져왔습니다


In [ ]:
금리2 = 금리.copy()

In [ ]:
금리.isna().sum()

CD91일     6
콜 금리      6
국고채 3년    6
회사채 3년    6
dtype: int64

In [ ]:
금리.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1196 entries, 2018.01.19 to 2022.11.22
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   CD91일   1190 non-null   object
 1   콜 금리    1190 non-null   object
 2   국고채 3년  1190 non-null   object
 3   회사채 3년  1190 non-null   object
dtypes: object(4)
memory usage: 79.0+ KB


In [ ]:
# 대출금리 가져온거 기존 데이터프레임에 합치기

def add_금리(df, 금리):

  # 수치형 변환
  for i in 금리.columns:
    금리[i] = 금리[i].astype('float')

  # 어차피 월별 평균으로 채울 것이기 때문에 널값은 삭제
  금리.dropna(axis=0, inplace=True)
  
  # 연월 만들어주기
  금리.reset_index(inplace=True)
  금리.rename(columns = {'index':'Date'}, inplace=True)
  
  금리['Date'] = 금리.apply(lambda x: x['Date'].replace('.','-'),axis=1)
  금리['연월'] = 금리.apply(lambda x: x['Date'][:7], axis=1)
  

  date = list(금리.groupby('연월')['CD91일'].mean().index)
  CD91일 = list(금리.groupby('연월')['CD91일'].mean().values)
  콜금리 = list(금리.groupby('연월')['콜 금리'].mean().values)
  국고채3년 = list(금리.groupby('연월')['국고채 3년'].mean().values)
  회사채3년 = list(금리.groupby('연월')['회사채 3년'].mean().values)


  금리_df = pd.DataFrame()
  금리_df['연월'] = date
  금리_df['CD91일'] = CD91일 #
  금리_df['콜금리'] = 콜금리 #
  금리_df['국고채3년'] = 국고채3년 #
  금리_df['회사채3년'] = 회사채3년 #

  # df['연월'] = df['결산년월'].astype('str')
  # df['연월'] = df.apply(lambda x:x['연월'][:4]+'-'+x['연월'][4:6], axis=1)

  df = pd.merge(df, 금리_df,how = 'left', on='연월')
  
  return df


df = add_금리(df, 금리)

df

,BIZ_NO,CMP_NM,BZ_TYP,CMP_SCL,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,휴폐업_여부,(주)여부,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,자본금,이익잉여금(결손금）,자본총계,매출액,판매비와관리비,영업이익（손실）,법인세비용차감전순손익,법인세비용,당기순이익(손실),기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,영업비율(%),금융비용대매출액비율(%,금융비용대부채비율(%),금융비용대총비용비율(%,부채비율(%),차입금의존도(%),자기자본비율(%),순운전자본비율(%),유동부채비율(%),비유동부채비율(%),부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),미수금,매출원가,무형자산,재고자산,K_score,K_부실여부판단,연월,환율,CD91일,콜금리,국고채3년,회사채3년
0,1138111739,한일가스산업,M,2,1,N,N,0,47712.0,N,31363,1,0,LP Gas 충전 및 판매업,etc,1,홍순철외 1명,43.0,0,1,20211231,2758846.0,1577971.0,11235042.0,5705994.0,13993889.0,6172590.0,6343124.0,12515714.0,300000.0,1178175.0,1478175.0,19738142.0,5176975.0,-979131.0,-812712.0,-812712.000,1.000000e+12,8.419,392.725,2.126800e+01,-4.961000e+00,-4.118000e+00,104.027,-2099711.928,1.049610e+02,3.230000e-01,1.000000e+12,0.303,8.467000e+02,2.416500e+01,1.056300e+01,-2.439500e+01,4.175820e+02,4.291190e+02,6.340900e+01,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,100333.000,15540298.0,2.500000e+04,5.917000e+04,3.012498e+06,91.793878,안전기업,2021-12,1183.480442,1.272174,1.022174,1.795652,2.404783
1,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,20211231,5562280.0,1443248.0,8236579.0,4351158.0,13798860.0,5247799.0,4002659.0,9250457.0,100000.0,4448402.0,4548402.0,14889722.0,765533.0,452145.0,391144.0,13605.000,3.775390e+05,3.881,32.238,4.448402e+03,8.178000e+00,3.037000e+00,2.536,97.398,-1.352845e+06,9.696300e+01,1.076000e+00,1.687,1.094000e+00,2.033780e+02,4.045100e+01,3.296200e+01,2.279000e+00,1.153770e+02,8.800100e+01,6.212700e+01,1.074000e+00,5.800000e+00,1.114000e+01,12.710,429334.0,1.367204e+07,2.080000e+02,2.324076e+06,11.838342,안전기업,2021-12,1183.480442,1.272174,1.022174,1.795652,2.404783
2,1318167565,도영운수,M,2,1,N,N,0,49212.0,N,31363,1,0,시내버스업,etc,1,김명화,149.0,0,1,20211231,1965055.0,1389692.0,4358581.0,4232380.0,6323635.0,3749079.0,1024357.0,4773436.0,200000.0,1350200.0,1550200.0,12582033.0,897080.0,-20771.0,-482373.0,37794.000,-4.823730e+05,-6.641,21.352,6.751000e+02,6.965000e+00,-1.650000e-01,-3.834,103.824,1.595847e+03,1.001650e+02,3.830000e-01,1.016,3.680000e-01,3.079240e+02,2.399300e+01,2.451400e+01,-2.821200e+01,2.418450e+02,6.607900e+01,3.793900e+01,1.925000e+00,1.000000e+12,9.850000e+00,67.045,520.0,1.170572e+07,9.170000e+02,3.012498e+06,10.720871,안전기업,2021-12,1183.480442,1.272174,1.022174,1.795652,2.404783
3,5048128251,중원산업,M,2,1,N,N,0,30391.0,N,31363,1,0,자동차 부품 제조,etc,1,김명구,75.0,0,1,20211231,12668826.0,5749707.0,15954138.0,13741069.0,28622964.0,9393776.0,3998392.0,13392168.0,50000.0,15180796.0,15230796.0,35437416.0,3491275.0,2124877.0,3044086.0,618646.000,2.425439e+06,9.158,49.543,2.836159e+04,1.584800e+01,5.996000e+00,6.844,91.682,-4.780308e+06,9.400400e+01,3.510000e-01,0.924,3.710000e-01,8.792800e+01,1.887400e+01,5.321200e+01,1.144200e+01,6.167600e+01,2.625200e+01,3.779100e+01,1.273000e+00,4.920100e+01,6.387000e+00,7.718,11688.0,2.982126e+07,4.746110e+05,1.035680e+06,16.822722,안전기업,2021-12,1183.480442,1.272174,1.022174,1.795652,2.404783
4,1098164260,카타나골프,M,2,1,N,N,0,46800.0,N,31363,1,0,골프용품 전반의 제조 및 수출업,etc,0,김홍득,3.0,0,1,20211231,10804850.0,137054.0,1146992.0,980971.0,11951841.0,6960351.0,4780000.0,11740351.0,100000.0,111490.0,211490.0,1172000.0,479947.0,12032.0,-146752.0,44086.000,-1.908380e+05,-0.139,0.933,1.114900e+02,4.197800e+01,1.027000e+00,-16.283,112.266,1.000000e+12,9.897300e+01,1.474700e+01,1.364,1.286700e+01,5.551256e+03,7.318600e+01,1.770000e+00,3.216700e+01,3.291102e+03,2.260154e+03,1.001736e+03,9.000000e-02,1.440000e+00,3.294000e+00,4.184,24272.0,6.800210e+05,8.159350e+05,3.012498e+06,-1.031020,판정유보기업,2021-12,1183.480442,1.272174,1.022174,1.795652,2.404783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,

In [ ]:
# df.to_csv('금리까지_붙인거.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/data/금리까지_붙인거.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35081 entries, 0 to 35080
Data columns (total 73 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   BIZ_NO           35081 non-null  int64  
 1   CMP_NM           35081 non-null  object 
 2   BZ_TYP           35081 non-null  object 
 3   CMP_SCL          35081 non-null  int64  
 4   HDOF_BR_GB       35081 non-null  int64  
 5   FR_IVST_CORP_YN  35081 non-null  object 
 6   VENT_YN          35081 non-null  object 
 7   LIST_CD          35081 non-null  int64  
 8   IND_CD1          35081 non-null  object 
 9   MDSCO_PRTC_YN    35081 non-null  object 
 10  HDOF_CMP_CD      35081 non-null  int64  
 11  ESTB_GB          35081 non-null  int64  
 12  LIST_DATE        35081 non-null  int64  
 13  MN_BIZ_CONT      35081 non-null  object 
 14  NATN_NM          35081 non-null  object 
 15  HOMEPAGE_URL     35081 non-null  int64  
 16  CEO_NM           35081 non-null  object 
 17  EMP_CNT     

In [ ]:
국제유가 = pd.read_csv('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/data/국제유가2022-11-20.csv')
국제유가.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2549 entries, 0 to 2548
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    2549 non-null   object 
 1   WTi     2549 non-null   float64
 2   Brent   2549 non-null   float64
 3   Dubai   2549 non-null   float64
dtypes: float64(3), object(1)
memory usage: 79.8+ KB


In [ ]:
국제유가

,Date,WTi,Brent,Dubai
0,2022-11-17,81.64,89.78,87.18
1,2022-11-16,85.59,92.86,88.06
2,2022-11-15,86.92,93.86,86.83
3,2022-11-14,85.87,93.14,91.04
4,2022-11-11,88.96,95.99,91.05
...,...,...,...,...
2544,2012-11-26,87.81,110.46,109.41
2545,2012-11-23,87.08,110.93,108.73
2546,2012-11-22,87.08,110.29,108.74
2547,2012-11-21,87.08,110.57,108.54


## 국제유가 데이터
- WTI
- Brent
- Dubai


In [ ]:
국제유가 = pd.read_csv('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/data/국제유가2022-11-20.csv')

def add_국제유가(df, 국제유가):

  # index인 Date컬럼 올리기
  국제유가 = 국제유가.reset_index()

  # 연월별로 평균내기(띄엄띄엄 있다.)
  국제유가['Date'] = 국제유가['Date'].astype('str')
  국제유가['연월'] = 국제유가.apply(lambda x: x['Date'][:7], axis=1)

  # 
  date = list(국제유가.groupby('연월')['WTi'].mean().index)
  WTi = list(국제유가.groupby('연월')['WTi'].mean().values)
  Brent = list(국제유가.groupby('연월')['Brent'].mean().values)
  Dubai = list(국제유가.groupby('연월')['Dubai'].mean().values)

  유가_df = pd.DataFrame()
  유가_df['연월'] = date
  유가_df['WTi'] = WTi
  유가_df['Brent'] = Brent
  유가_df['Dubai'] = Dubai
  

  df = pd.merge(df, 유가_df, how='left', on='연월')
  return df


print(f'기존 df의 shape:{df.shape}')
df = add_국제유가(df, 국제유가)
print(f'국제유가 컬럼 추가 후 df의 shape:{df.shape}')

기존 df의 shape:(35081, 73)
국제유가 컬럼 추가 후 df의 shape:(35081, 76)


## 발탁운임지수(BDI; Baltic Dry Index)
- 발탁운임지수란 석탄, 철광석, 시멘트, 곡물 등 원자재를 싣고 26개 주요 해상운송경로를 지나는 선적량 15천 톤 이상 선박의 화물운임과 용선료 등을 종합해 산정하는 지수로, 배달이 원자재를 '얼마나 많이' 싣고, '얼마나 자주' 돌아다니는지를 알려주는 지표다.

In [ ]:
BDI

,날짜,종가,오픈,고가,저가,거래량,변동 %
0,2022- 11- 18,"1,189.00","1,189.00","1,189.00","1,189.00",NaN,-3.18%
1,2022- 11- 17,"1,228.00","1,228.00","1,228.00","1,228.00",NaN,-4.66%
2,2022- 11- 16,"1,288.00","1,288.00","1,288.00","1,288.00",NaN,-0.92%
3,2022- 11- 15,"1,300.00","1,300.00","1,300.00","1,300.00",NaN,-1.89%
4,2022- 11- 14,"1,325.00","1,325.00","1,325.00","1,325.00",NaN,-2.21%
...,...,...,...,...,...,...,...
2716,2012- 01- 03,"1,624.00","1,624.00","1,624.00","1,624.00",NaN,-6.56%
2717,2011- 12- 23,"1,738.00","1,738.00","1,738.00","1,738.00",NaN,-2.74%
2718,2011- 12- 22,"1,787.00","1,787.00","1,787.00","1,787.00",NaN,-3.72%
2719,2011- 12- 21,"1,856.00","1,856.00","1,856.00","1,856.00",NaN,-1.17%


In [ ]:
BDI = pd.read_csv('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/data/Baltic Dry Index 내역.csv')
BDI

def add_발탁운임지수(df, BDI):

  # 종가만 취급할 것임.
  BDI = BDI[['날짜', '종가']]

  # 연월 만들어주기
  BDI['날짜'] = BDI.apply(lambda x: x['날짜'].replace(' ', ''), axis=1)
  BDI['연월'] = BDI.apply(lambda x: x['날짜'][:7], axis=1)
  
  # 종가에서 , 지우기
  BDI['종가'] = BDI['종가'].astype('str')
  BDI['종가'] = BDI.apply(lambda x: x['종가'].replace(',', ''), axis=1)
  BDI['종가'] = BDI['종가'].astype('float')

  # 월별 평균값 만들어주기
  date = list(BDI.groupby('연월')['종가'].mean().index)
  BDI_price = list(BDI.groupby('연월')['종가'].mean().values)

  BDI_df = pd.DataFrame()
  BDI_df['연월'] = date
  BDI_df['BDI'] = BDI_price

  df = pd.merge(df, BDI_df, how='left', on='연월')
  return df


print(f'기존 df의 shape:{df.shape}')
df = add_발탁운임지수(df, BDI)
print(f'발탁운임지수 컬럼 추가 후 df의 shape:{df.shape}')

기존 df의 shape:(35081, 76)
발탁운임지수 컬럼 추가 후 df의 shape:(35081, 77)


## 국내수입단가


In [ ]:
수입단가 = pd.read_csv('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/data/국내수입단가2022-11-20.csv')
수입단가

,Date,Price,grain_name,supply,amount
0,2022-09-01,0.390710,Corn,1038115377,405602422
1,2022-09-01,0.784476,Soybean,83936998,65846573
2,2022-08-01,0.744195,Soybean,159308611,118556633
3,2022-08-01,0.401754,Corn,968492559,389095498
4,2022-08-01,0.404278,Wheat,289704113,117120967
...,...,...,...,...,...
813,2000-02-01,0.211650,Soybean,101373508,21455698
814,2000-02-01,0.105457,Corn,627817347,66207559
815,2000-01-01,0.213490,Soybean,107643096,22980736
816,2000-01-01,0.105546,Corn,663585480,70038667


In [ ]:
수입단가.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 818 entries, 0 to 817
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Date        818 non-null    object 
 1   Price       818 non-null    float64
 2   grain_name  818 non-null    object 
 3   supply      818 non-null    int64  
 4   amount      818 non-null    int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 32.1+ KB


In [ ]:
수입단가 = pd.read_csv('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/data/국내수입단가2022-11-20.csv')
수입단가

def add_수입단가(df, 수입단가):
  

  수입단가['연월'] = 수입단가.apply(lambda x: x['Date'][:7], axis=1)

  #date = list(수입단가.groupby('연월')['Price'].mean().index)

  Corn_df = 수입단가[수입단가['grain_name']=='Corn']
  Soybean_df = 수입단가[수입단가['grain_name']=='Soybean']
  Wheat_df = 수입단가[수입단가['grain_name']=='Wheat']

  Corn_dfdf = pd.DataFrame(Corn_df.groupby('연월')['Price'].mean())
  Soybean_dfdf = pd.DataFrame(Soybean_df.groupby('연월')['Price'].mean())
  Wheat_dfdf = pd.DataFrame(Wheat_df.groupby('연월')['Price'].mean())

  # Corn_dfdf.columns = ['연월', 'Corn_price']
  # Soybean_dfdf.columns = ['연월', 'Soybean_price']
  # Wheat_dfdf.columns = ['연월', 'Wheat_price']

  df = pd.merge(df, Corn_dfdf, how='left', on='연월')
  df.rename(columns = {'Price':'Corn_price'}, inplace=True)

  df = pd.merge(df, Soybean_dfdf,how='left', on='연월')
  df.rename(columns = {'Price':'Soybean_price'}, inplace=True)
  
  df = pd.merge(df, Wheat_dfdf,how='left', on='연월')
  df.rename(columns = {'Price':'Wheat_price'}, inplace=True)

  return df

print(f'기존 df의 shape:{df.shape}')
df = add_수입단가(df, 수입단가)
print(f'국제유가 컬럼 추가 후 df의 shape:{df.shape}')

기존 df의 shape:(35081, 77)
국제유가 컬럼 추가 후 df의 shape:(35081, 80)


## Food Price Index

In [ ]:
Food_price_index= pd.read_csv('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/data/Food_price_indices_data_nov491.csv')

# 연월 만드는 함수
def make_연월(x):
  if x[:3]=='Jan':
    x = '20'+x[4:]+'-'+'01'
  if x[:3]=='Feb':
    x = '20'+x[4:]+'-'+'02'
  if x[:3]=='Mar':
    x = '20'+x[4:]+'-'+'03'
  if x[:3]=='Apr':
    x = '20'+x[4:]+'-'+'04'
  if x[:3]=='May':
    x = '20'+x[4:]+'-'+'05'
  if x[:3]=='Jun':
    x = '20'+x[4:]+'-'+'06'
  if x[:3]=='Jul':
    x = '20'+x[4:]+'-'+'07'
  if x[:3]=='Aug':
    x = '20'+x[4:]+'-'+'08'
  if x[:3]=='Sep':
    x = '20'+x[4:]+'-'+'09'
  if x[:3]=='Oct':
    x = '20'+x[4:]+'-'+'10'
  if x[:3]=='Nov':
    x = '20'+x[4:]+'-'+'11'
  if x[:3]=='Dec':
    x = '20'+x[4:]+'-'+'12'

  return x


def add_food_price_index(df, Food_price_index):
    Food_price_index = Food_price_index.loc[list(Food_price_index[Food_price_index['Date']=='Jan-18'].index).pop():, :] # 18년 1월1일부터 불러오기
    Food_price_index['Date'] = Food_price_index['Date'].apply(make_연월)

    Food_price_index = Food_price_index[['Date', 'Food Price Index']]
    Food_price_index.columns = ['연월', 'Food_Price_Index']
    df = pd.merge(df, Food_price_index, how='left', on='연월')

    return df


print(f'기존 df의 shape:{df.shape}')
df = add_food_price_index(df, Food_price_index)
print(f'Food_price_index 컬럼 추가 후 df의 shape:{df.shape}')

기존 df의 shape:(35081, 80)
Food_price_index 컬럼 추가 후 df의 shape:(35081, 81)


## Sugar-Cotton

In [ ]:
suggar_cotton = pd.read_csv('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/data/SUGAR-COTTON선물가격2022-11-20.csv')
suggar_cotton

def add_sugar_cotton(df, suggar_cotton):
  
  sugar_df = suggar_cotton[suggar_cotton['Type']=='Sugar']
  cotton_df = suggar_cotton[suggar_cotton['Type']=='Cotton']

  sugar_df['연월'] = sugar_df['Date'].apply(lambda x: x[:7])
  cotton_df['연월'] = cotton_df['Date'].apply(lambda x: x[:7])

  sugar_dfdf =  pd.DataFrame(sugar_df.groupby('연월')['Settlement'].mean())
  cotton_dfdf =  pd.DataFrame(cotton_df.groupby('연월')['Settlement'].mean())

  df = pd.merge(df, sugar_dfdf, how='left', on='연월')
  df.rename(columns = {'Settlement':'Sugar_price'}, inplace=True)

  df = pd.merge(df, cotton_dfdf, how='left', on='연월')
  df.rename(columns = {'Settlement':'Cotton_price'}, inplace=True) 

  return df

print(f'기존 df의 shape:{df.shape}')
df = add_sugar_cotton(df, suggar_cotton)
print(f'Suggar & Cotton 컬럼 추가 후 df의 shape:{df.shape}')

기존 df의 shape:(35081, 81)
Suggar & Cotton 컬럼 추가 후 df의 shape:(35081, 83)


In [ ]:
df.to_csv('일단1치병합.csv', index=False)

## 금, 구리, 은
- 퀀들에서 데이터 가져오기
- 퀀들 가입 후 API KEY 받기

In [ ]:
!pip install quandl

In [ ]:
import quandl

QUANDL_KEY = 'Z2htjZxAdKrqR8V9xPgg'
quandl.ApiConfig.api_key = QUANDL_KEY
df_gold = quandl.get(dataset='LBMA/GOLD', start_date = '2018-01-01')
df_gold.reset_index(inplace=True)
df_gold = df_gold[['Date', 'USD (AM)']]
df_gold

df_silver = quandl.get(dataset='LBMA/SILVER', start_date = '2018-01-01')
df_silver.reset_index(inplace=True)
df_silver = df_silver[['Date', 'USD']]
df_silver

df_copper = quandl.get("CHRIS/CME_HG10", start_date='2018-01-01')
df_copper.reset_index(inplace=True)
df_copper = df_copper[['Date', 'Settle']]





In [ ]:
df_copper.tail(10)

,Date,Settle
869,2021-06-16,4.4020
870,2021-06-17,4.1975
871,2021-06-18,4.1750
872,2021-06-21,4.1905
873,2021-06-22,4.2205
874,2021-06-23,4.3280
875,2021-06-24,4.3095
876,2021-06-25,4.2910
877,2021-06-28,4.2825
878,2021-06-29,4.2525


In [ ]:
def add_금은구리(df, df_gold, df_silver, df_copper):
  df_gold['Date'] = df_gold['Date'].astype('str')
  df_gold['연월'] = df_gold.apply(lambda x: x['Date'][:7], axis=1)
  df_gold.rename(columns = {'USD (AM)': 'gold_price'}, inplace=True)
  
  df_silver['Date'] = df_silver['Date'].astype('str')
  df_silver['연월'] = df_silver.apply(lambda x: x['Date'][:7], axis=1)
  df_silver.rename(columns = {'USD':'silver_price'}, inplace=True)

  df_copper['Date'] = df_copper['Date'].astype('str')
  df_copper['연월'] = df_copper.apply(lambda x: x['Date'][:7], axis=1)
  df_copper.rename(columns={'Settle':'copper_price'}, inplace=True)

  gold_dfdf = pd.DataFrame(df_gold.groupby('연월')['gold_price'].mean())
  silver_dfdf = pd.DataFrame(df_silver.groupby('연월')['silver_price'].mean())
  copper_dfdf = pd.DataFrame(df_copper.groupby('연월')['copper_price'].mean())


  df = pd.merge(df, gold_dfdf, how = 'left', on='연월')
  df = pd.merge(df, silver_dfdf, how = 'left', on='연월')
  df = pd.merge(df, copper_dfdf, how = 'left', on='연월')

  return df


print(f'기존 df의 shape:{df.shape}')
df = add_금은구리(df, df_gold, df_silver, df_copper)
print(f'금,은,구리 컬럼 추가 후 df의 shape:{df.shape}')


기존 df의 shape:(35081, 83)
금,은,구리 컬럼 추가 후 df의 shape:(35081, 86)


In [ ]:
df

,BIZ_NO,CMP_NM,BZ_TYP,CMP_SCL,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,휴폐업_여부,(주)여부,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,자본금,이익잉여금(결손금）,자본총계,매출액,판매비와관리비,영업이익（손실）,법인세비용차감전순손익,법인세비용,당기순이익(손실),기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,영업비율(%),금융비용대매출액비율(%,금융비용대부채비율(%),금융비용대총비용비율(%,부채비율(%),차입금의존도(%),자기자본비율(%),순운전자본비율(%),유동부채비율(%),비유동부채비율(%),부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),미수금,매출원가,무형자산,재고자산,K_score,K_부실여부판단,연월,환율,CD91일,콜금리,국고채3년,회사채3년,WTi,Brent,Dubai,BDI,Corn_price,Soybean_price,Wheat_price,Food_Price_Index,Sugar_price,Cotton_price,gold_price,silver_price,copper_price
0,1138111739,한일가스산업,M,2,1,N,N,0,47712.0,N,31363,1,0,LP Gas 충전 및 판매업,etc,1,홍순철외 1명,43.0,0,1,20211231,2758846.0,1577971.0,11235042.0,5705994.0,13993889.0,6172590.0,6343124.0,12515714.0,300000.0,1178175.0,1478175.0,19738142.0,5176975.0,-979131.0,-812712.0,-812712.000,1.000000e+12,8.419,392.725,2.126800e+01,-4.961000e+00,-4.118000e+00,104.027,-2099711.928,1.049610e+02,3.230000e-01,1.000000e+12,0.303,8.467000e+02,2.416500e+01,1.056300e+01,-2.439500e+01,4.175820e+02,4.291190e+02,6.340900e+01,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,100333.000,15540298.0,2.500000e+04,5.917000e+04,3.012498e+06,91.793878,안전기업,2021-12,1183.480442,1.272174,1.022174,1.795652,2.404783,71.781304,74.803913,73.257391,2832.111111,0.320083,0.739519,0.338347,133.7,19.146818,108.876667,1788.633333,22.468095,NaN
1,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,20211231,5562280.0,1443248.0,8236579.0,4351158.0,13798860.0,5247799.0,4002659.0,9250457.0,100000.0,4448402.0,4548402.0,14889722.0,765533.0,452145.0,391144.0,13605.000,3.775390e+05,3.881,32.238,4.448402e+03,8.178000e+00,3.037000e+00,2.536,97.398,-1.352845e+06,9.696300e+01,1.076000e+00,1.687,1.094000e+00,2.033780e+02,4.045100e+01,3.296200e+01,2.279000e+00,1.153770e+02,8.800100e+01,6.212700e+01,1.074000e+00,5.800000e+00,1.114000e+01,12.710,429334.0,1.367204e+07,2.080000e+02,2.324076e+06,11.838342,안전기업,2021-12,1183.480442,1.272174,1.022174,1.795652,2.404783,71.781304,74.803913,73.257391,2832.111111,0.320083,0.739519,0.338347,133.7,19.146818,108.876667,1788.633333,22.468095,NaN
2,1318167565,도영운수,M,2,1,N,N,0,49212.0,N,31363,1,0,시내버스업,etc,1,김명화,149.0,0,1,20211231,1965055.0,1389692.0,4358581.0,4232380.0,6323635.0,3749079.0,1024357.0,4773436.0,200000.0,1350200.0,1550200.0,12582033.0,897080.0,-20771.0,-482373.0,37794.000,-4.823730e+05,-6.641,21.352,6.751000e+02,6.965000e+00,-1.650000e-01,-3.834,103.824,1.595847e+03,1.001650e+02,3.830000e-01,1.016,3.680000e-01,3.079240e+02,2.399300e+01,2.451400e+01,-2.821200e+01,2.418450e+02,6.607900e+01,3.793900e+01,1.925000e+00,1.000000e+12,9.850000e+00,67.045,520.0,1.170572e+07,9.170000e+02,3.012498e+06,10.720871,안전기업,2021-12,1183.480442,1.272174,1.022174,1.795652,2.404783,71.781304,74.803913,73.257391,2832.111111,0.320083,0.739519,0.338347,133.7,19.146818,108.876667,1788.633333,22.468095,NaN
3,5048128251,중원산업,M,2,1,N,N,0,30391.0,N,31363,1,0,자동차 부품 제조,etc,1,김명구,75.0,0,1,20211231,12668826.0,5749707.0,15954138.0,13741069.0,28622964.0,9393776.0,3998392.0,13392168.0,50000.0,15180796.0,15230796.0,35437416.0,3491275.0,2124877.0,3044086.0,618646.000,2.425439e+06,9.158,49.543,2.836159e+04,1.584800e+01,5.996000e+00,6.844,91.682,-4.780308e+06,9.400400e+01,3.510000e-01,0.924,3.710000e-01,8.792800e+01,1.887400e+01,5.321200e+01,1.144200e+01,6.167600e+01,2.625200e+01,3.779100e+01,1.273000e+00,4.920100e+01,6.387000e+00,7.718,11688.0,2.982126e+07,4.746110e+05,1.035680e+06,16.822722,안전기업,2021-12,1183.480442,1.272174,1.022174,1.795652,2.404783,71.781304,74.803913,73.257391,2832.111111,0.320083,0.739519,0.338347,133.7,19.146818,108.876667,1788.633333,22.468095,NaN
4,1098164260,카타나골프,M,2,1,N,N,0,46800.0,N,31363,1,0,골프용품 전반의 제조 및 수출업,etc,0,김홍득,3.0,0,1,20211231,10804850.0,137054.0,1146992.0,980971.0,11951841.0,6960351.0

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35081 entries, 0 to 35080
Data columns (total 86 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   BIZ_NO            35081 non-null  int64  
 1   CMP_NM            35081 non-null  object 
 2   BZ_TYP            35081 non-null  object 
 3   CMP_SCL           35081 non-null  int64  
 4   HDOF_BR_GB        35081 non-null  int64  
 5   FR_IVST_CORP_YN   35081 non-null  object 
 6   VENT_YN           35081 non-null  object 
 7   LIST_CD           35081 non-null  int64  
 8   IND_CD1           35081 non-null  object 
 9   MDSCO_PRTC_YN     35081 non-null  object 
 10  HDOF_CMP_CD       35081 non-null  int64  
 11  ESTB_GB           35081 non-null  int64  
 12  LIST_DATE         35081 non-null  int64  
 13  MN_BIZ_CONT       35081 non-null  object 
 14  NATN_NM           35081 non-null  object 
 15  HOMEPAGE_URL      35081 non-null  int64  
 16  CEO_NM            35081 non-null  object

In [ ]:
# 구리 널값 너무 많아서 드롭
df.drop('copper_price', axis=1, inplace=True)

## 크롤링 데이터 병합

In [ ]:
tmp = pd.read_csv('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/data/real_baseline_df.csv')
crawling_1 = pd.read_csv('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/data/HR_data.csv')
crawling_2 = pd.read_csv('/content/drive/MyDrive/나이스디앤비/넘블 챌린지 데이터/data/HR_data_add.csv', encoding='cp949')

In [ ]:
# 크롤링한 데이터 전처리
crawling_df = pd.DataFrame()
crawling_df['BIZ_NO'] = tmp['BIZ_NO']

crawling_df = pd.concat([crawling_df, crawling_2], axis=1)
crawling_df.drop('Unnamed: 0', axis=1,inplace=True)
crawling_df



,BIZ_NO,avg_wage,avg_wage_percent,total_employee,turnover_rate,turnover_rate_ranking,employee_growth_rate,employee_growth_rate_ranking
0,1138111739,"2,861만원",연봉상위 78%,18명,26%,상위 17%,5%,상위 100%
1,2078132193,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1318167565,"1,953만원",연봉상위 100%,3명,- %,상위 - %,- %,상위 - %
3,5048128251,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1098164260,"1,906만원",연봉상위 100%,3명,- %,상위 - %,- %,상위 - %
...,...,...,...,...,...,...,...,...
35081,4808501629,"2,900만원",연봉상위 100%,6명,107%,상위 93%,178%,상위 4%
35082,8018501578,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35083,4118523419,"1,700만원",연봉상위 100%,3명,133%,상위 84%,67%,상위 86%
35084,1738102533,"2,170만원",연봉상위 100%,88명,- %,상위 - %,- %,상위 - %


In [ ]:
import re
for i in crawling_df.columns[1:]:
  crawling_df[i] = crawling_df[i].apply(lambda x: re.sub(r'[^0-9]', '', str(x))) # 문자열 제거하고 숫자만 반환

In [ ]:
crawling_df.replace('',np.nan, inplace=True)
crawling_df

,BIZ_NO,avg_wage,avg_wage_percent,total_employee,turnover_rate,turnover_rate_ranking,employee_growth_rate,employee_growth_rate_ranking
0,1138111739,2861,78,18,26,17,5,100
1,2078132193,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1318167565,1953,100,3,NaN,NaN,NaN,NaN
3,5048128251,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1098164260,1906,100,3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
35081,4808501629,2900,100,6,107,93,178,4
35082,8018501578,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35083,4118523419,1700,100,3,133,84,67,86
35084,1738102533,2170,100,88,NaN,NaN,NaN,NaN


In [ ]:
crawling_df.isna().sum()

BIZ_NO                              0
avg_wage                        23052
avg_wage_percent                23052
total_employee                  23052
turnover_rate                   28319
turnover_rate_ranking           28319
employee_growth_rate            28318
employee_growth_rate_ranking    28318
dtype: int64

In [ ]:
# object -> float형으로
for i in crawling_df.columns[1:]:
  crawling_df[i] = crawling_df[i].astype('float')

# 평균값으로 채운거
crawling_df_mean = crawling_df.copy()
for i in crawling_df_mean.columns[1:]:
  crawling_df_mean[i] = crawling_df_mean[i].fillna(crawling_df_mean[i].mean())

# 최빈값으로 채운거
crawling_df_mode = crawling_df.copy()
for i in crawling_df_mode.columns[1:]:
  crawling_df_mode[i] = crawling_df_mode[i].fillna(crawling_df_mode[i].mode())

df_mean으로채운거 = df.copy()
df_mode으로채운거 = df.copy()

df_mean으로채운거 = pd.merge(df_mean으로채운거, crawling_df_mean, on='BIZ_NO')
df_mode으로채운거 = pd.merge(df_mode으로채운거, crawling_df_mode, on='BIZ_NO')

In [ ]:
df_mean으로채운거

,BIZ_NO,CMP_NM,BZ_TYP,CMP_SCL,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,휴폐업_여부,(주)여부,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,자본금,이익잉여금(결손금）,자본총계,매출액,판매비와관리비,영업이익（손실）,법인세비용차감전순손익,법인세비용,당기순이익(손실),기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,영업비율(%),금융비용대매출액비율(%,금융비용대부채비율(%),금융비용대총비용비율(%,부채비율(%),차입금의존도(%),자기자본비율(%),순운전자본비율(%),유동부채비율(%),비유동부채비율(%),부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),미수금,매출원가,무형자산,재고자산,K_score,K_부실여부판단,연월,환율,CD91일,콜금리,국고채3년,회사채3년,WTi,Brent,Dubai,BDI,Corn_price,Soybean_price,Wheat_price,Food_Price_Index,Sugar_price,Cotton_price,gold_price,silver_price,avg_wage,avg_wage_percent,total_employee,turnover_rate,turnover_rate_ranking,employee_growth_rate,employee_growth_rate_ranking
0,1138111739,한일가스산업,M,2,1,N,N,0,47712.0,N,31363,1,0,LP Gas 충전 및 판매업,etc,1,홍순철외 1명,43.0,0,1,20211231,2758846.0,1577971.0,11235042.0,5705994.0,13993889.0,6172590.0,6343124.0,12515714.0,300000.0,1178175.0,1478175.0,19738142.0,5176975.0,-979131.0,-812712.0,-812712.000,1.000000e+12,8.419,392.725,2.126800e+01,-4.961000e+00,-4.118000e+00,104.027,-2099711.928,1.049610e+02,3.230000e-01,1.000000e+12,0.303,8.467000e+02,2.416500e+01,1.056300e+01,-2.439500e+01,4.175820e+02,4.291190e+02,6.340900e+01,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,100333.000,15540298.0,2.500000e+04,5.917000e+04,3.012498e+06,91.793878,안전기업,2021-12,1183.480442,1.272174,1.022174,1.795652,2.404783,71.781304,74.803913,73.257391,2832.111111,0.320083,0.739519,0.338347,133.7,19.146818,108.876667,1788.633333,22.468095,2861.000000,78.00000,18.000000,26.000000,17.000000,5.000000,100.000000
1,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,20211231,5562280.0,1443248.0,8236579.0,4351158.0,13798860.0,5247799.0,4002659.0,9250457.0,100000.0,4448402.0,4548402.0,14889722.0,765533.0,452145.0,391144.0,13605.000,3.775390e+05,3.881,32.238,4.448402e+03,8.178000e+00,3.037000e+00,2.536,97.398,-1.352845e+06,9.696300e+01,1.076000e+00,1.687,1.094000e+00,2.033780e+02,4.045100e+01,3.296200e+01,2.279000e+00,1.153770e+02,8.800100e+01,6.212700e+01,1.074000e+00,5.800000e+00,1.114000e+01,12.710,429334.0,1.367204e+07,2.080000e+02,2.324076e+06,11.838342,안전기업,2021-12,1183.480442,1.272174,1.022174,1.795652,2.404783,71.781304,74.803913,73.257391,2832.111111,0.320083,0.739519,0.338347,133.7,19.146818,108.876667,1788.633333,22.468095,2618.076201,82.77871,28.742563,59.190483,55.080538,47.626773,59.519504
2,1318167565,도영운수,M,2,1,N,N,0,49212.0,N,31363,1,0,시내버스업,etc,1,김명화,149.0,0,1,20211231,1965055.0,1389692.0,4358581.0,4232380.0,6323635.0,3749079.0,1024357.0,4773436.0,200000.0,1350200.0,1550200.0,12582033.0,897080.0,-20771.0,-482373.0,37794.000,-4.823730e+05,-6.641,21.352,6.751000e+02,6.965000e+00,-1.650000e-01,-3.834,103.824,1.595847e+03,1.001650e+02,3.830000e-01,1.016,3.680000e-01,3.079240e+02,2.399300e+01,2.451400e+01,-2.821200e+01,2.418450e+02,6.607900e+01,3.793900e+01,1.925000e+00,1.000000e+12,9.850000e+00,67.045,520.0,1.170572e+07,9.170000e+02,3.012498e+06,10.720871,안전기업,2021-12,1183.480442,1.272174,1.022174,1.795652,2.404783,71.781304,74.803913,73.257391,2832.111111,0.320083,0.739519,0.338347,133.7,19.146818,108.876667,1788.633333,22.468095,1953.000000,100.00000,3.000000,59.190483,55.080538,47.626773,59.519504
3,5048128251,중원산업,M,2,1,N,N,0,30391.0,N,31363,1,0,자동차 부품 제조,etc,1,김명구,75.0,0,1,20211231,12668826.0,5749707.0,15954138.0,13741069.0,28622964.0,9393776.0,3998392.0,13392168.0,50000.0,15180796.0,15230796.0,35437416.0,3491275.0,2124877.0,3044086.0,618646.000,2.425439e+06,9.158,49.543,2.836159e+04,1.584800e+01,5.996000e+00,6.844,91.682,-4.780308e+06,9.400400e+01,3.510000e-01,0.924,3.710000e-01,8.792800e+01,1.887400e+01,5.321200e+01,1.144200e+01,6.167600e+01,2.625200e+01,3.779100e+01,1.273000e+00,4.920100e+01,6.387000e+00,7.718,11688.0,2.982126e+07,4.746110e+05,1.035680e+06,16.822722,안전기업,2021-12,1183.48044

In [ ]:
df_mean으로채운거.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35081 entries, 0 to 35080
Data columns (total 92 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   BIZ_NO                        35081 non-null  int64  
 1   CMP_NM                        35081 non-null  object 
 2   BZ_TYP                        35081 non-null  object 
 3   CMP_SCL                       35081 non-null  int64  
 4   HDOF_BR_GB                    35081 non-null  int64  
 5   FR_IVST_CORP_YN               35081 non-null  object 
 6   VENT_YN                       35081 non-null  object 
 7   LIST_CD                       35081 non-null  int64  
 8   IND_CD1                       35081 non-null  object 
 9   MDSCO_PRTC_YN                 35081 non-null  object 
 10  HDOF_CMP_CD                   35081 non-null  int64  
 11  ESTB_GB                       35081 non-null  int64  
 12  LIST_DATE                     35081 non-null  int64  
 13  M

In [ ]:
df_mode으로채운거.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35081 entries, 0 to 35080
Data columns (total 92 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   BIZ_NO                        35081 non-null  int64  
 1   CMP_NM                        35081 non-null  object 
 2   BZ_TYP                        35081 non-null  object 
 3   CMP_SCL                       35081 non-null  int64  
 4   HDOF_BR_GB                    35081 non-null  int64  
 5   FR_IVST_CORP_YN               35081 non-null  object 
 6   VENT_YN                       35081 non-null  object 
 7   LIST_CD                       35081 non-null  int64  
 8   IND_CD1                       35081 non-null  object 
 9   MDSCO_PRTC_YN                 35081 non-null  object 
 10  HDOF_CMP_CD                   35081 non-null  int64  
 11  ESTB_GB                       35081 non-null  int64  
 12  LIST_DATE                     35081 non-null  int64  
 13  M

In [ ]:
df_mean으로채운거.isna().sum().sum()

0

In [ ]:
# df_mean으로채운거.replace(np.inf:np.nan, inplace=True)
# df_mean으로채운거.replace(-np.inf:np.nan, inplace=True)
# df_mean으로채운거
df_mean으로채운거 = df_mean으로채운거.replace([np.inf, -np.inf], np.nan) 
df_mean으로채운거.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35081 entries, 0 to 35080
Data columns (total 92 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   BIZ_NO                        35081 non-null  int64  
 1   CMP_NM                        35081 non-null  object 
 2   BZ_TYP                        35081 non-null  object 
 3   CMP_SCL                       35081 non-null  int64  
 4   HDOF_BR_GB                    35081 non-null  int64  
 5   FR_IVST_CORP_YN               35081 non-null  object 
 6   VENT_YN                       35081 non-null  object 
 7   LIST_CD                       35081 non-null  int64  
 8   IND_CD1                       35081 non-null  object 
 9   MDSCO_PRTC_YN                 35081 non-null  object 
 10  HDOF_CMP_CD                   35081 non-null  int64  
 11  ESTB_GB                       35081 non-null  int64  
 12  LIST_DATE                     35081 non-null  int64  
 13  M

In [ ]:
df_mean으로채운거.dropna(axis=0, inplace=True)
df_mean으로채운거

,BIZ_NO,CMP_NM,BZ_TYP,CMP_SCL,HDOF_BR_GB,FR_IVST_CORP_YN,VENT_YN,LIST_CD,IND_CD1,MDSCO_PRTC_YN,HDOF_CMP_CD,ESTB_GB,LIST_DATE,MN_BIZ_CONT,NATN_NM,HOMEPAGE_URL,CEO_NM,EMP_CNT,휴폐업_여부,(주)여부,결산년월,유동자산,매출채권,비유동자산,유형자산,자산총계,유동부채,비유동부채,부 채 총 계,자본금,이익잉여금(결손금）,자본총계,매출액,판매비와관리비,영업이익（손실）,법인세비용차감전순손익,법인세비용,당기순이익(손실),기업순이익률(%),유보액/총자산(%),유보액/납입자본(%),매출액총이익률(%),매출액영업이익률(%),매출액순이익률(%),수지비율(%),경상수지비율,영업비율(%),금융비용대매출액비율(%,금융비용대부채비율(%),금융비용대총비용비율(%,부채비율(%),차입금의존도(%),자기자본비율(%),순운전자본비율(%),유동부채비율(%),비유동부채비율(%),부채총계대 매출액(%),총자본회전율(회),재고자산회전율(회),매출채권회전율(회),매입채무회전율(회),미수금,매출원가,무형자산,재고자산,K_score,K_부실여부판단,연월,환율,CD91일,콜금리,국고채3년,회사채3년,WTi,Brent,Dubai,BDI,Corn_price,Soybean_price,Wheat_price,Food_Price_Index,Sugar_price,Cotton_price,gold_price,silver_price,avg_wage,avg_wage_percent,total_employee,turnover_rate,turnover_rate_ranking,employee_growth_rate,employee_growth_rate_ranking
0,1138111739,한일가스산업,M,2,1,N,N,0,47712.0,N,31363,1,0,LP Gas 충전 및 판매업,etc,1,홍순철외 1명,43.0,0,1,20211231,2758846.0,1577971.0,11235042.0,5705994.0,13993889.0,6172590.0,6343124.0,12515714.0,300000.0,1178175.0,1478175.0,19738142.0,5176975.0,-979131.0,-812712.0,-812712.000,1.000000e+12,8.419,392.725,2.126800e+01,-4.961000e+00,-4.118000e+00,104.027,-2099711.928,1.049610e+02,3.230000e-01,1.000000e+12,0.303,8.467000e+02,2.416500e+01,1.056300e+01,-2.439500e+01,4.175820e+02,4.291190e+02,6.340900e+01,1.000000e+12,1.000000e+12,1.000000e+12,1.000000e+12,100333.000,15540298.0,2.500000e+04,5.917000e+04,3.012498e+06,91.793878,안전기업,2021-12,1183.480442,1.272174,1.022174,1.795652,2.404783,71.781304,74.803913,73.257391,2832.111111,0.320083,0.739519,0.338347,133.7,19.146818,108.876667,1788.633333,22.468095,2861.000000,78.00000,18.000000,26.000000,17.000000,5.000000,100.000000
1,2078132193,엠피아이,M,2,1,N,N,0,etc,N,31363,1,0,가죽 및 대용가죽제품 제조업,etc,1,이스테판상수외 1명,35.0,0,1,20211231,5562280.0,1443248.0,8236579.0,4351158.0,13798860.0,5247799.0,4002659.0,9250457.0,100000.0,4448402.0,4548402.0,14889722.0,765533.0,452145.0,391144.0,13605.000,3.775390e+05,3.881,32.238,4.448402e+03,8.178000e+00,3.037000e+00,2.536,97.398,-1.352845e+06,9.696300e+01,1.076000e+00,1.687,1.094000e+00,2.033780e+02,4.045100e+01,3.296200e+01,2.279000e+00,1.153770e+02,8.800100e+01,6.212700e+01,1.074000e+00,5.800000e+00,1.114000e+01,12.710,429334.0,1.367204e+07,2.080000e+02,2.324076e+06,11.838342,안전기업,2021-12,1183.480442,1.272174,1.022174,1.795652,2.404783,71.781304,74.803913,73.257391,2832.111111,0.320083,0.739519,0.338347,133.7,19.146818,108.876667,1788.633333,22.468095,2618.076201,82.77871,28.742563,59.190483,55.080538,47.626773,59.519504
2,1318167565,도영운수,M,2,1,N,N,0,49212.0,N,31363,1,0,시내버스업,etc,1,김명화,149.0,0,1,20211231,1965055.0,1389692.0,4358581.0,4232380.0,6323635.0,3749079.0,1024357.0,4773436.0,200000.0,1350200.0,1550200.0,12582033.0,897080.0,-20771.0,-482373.0,37794.000,-4.823730e+05,-6.641,21.352,6.751000e+02,6.965000e+00,-1.650000e-01,-3.834,103.824,1.595847e+03,1.001650e+02,3.830000e-01,1.016,3.680000e-01,3.079240e+02,2.399300e+01,2.451400e+01,-2.821200e+01,2.418450e+02,6.607900e+01,3.793900e+01,1.925000e+00,1.000000e+12,9.850000e+00,67.045,520.0,1.170572e+07,9.170000e+02,3.012498e+06,10.720871,안전기업,2021-12,1183.480442,1.272174,1.022174,1.795652,2.404783,71.781304,74.803913,73.257391,2832.111111,0.320083,0.739519,0.338347,133.7,19.146818,108.876667,1788.633333,22.468095,1953.000000,100.00000,3.000000,59.190483,55.080538,47.626773,59.519504
3,5048128251,중원산업,M,2,1,N,N,0,30391.0,N,31363,1,0,자동차 부품 제조,etc,1,김명구,75.0,0,1,20211231,12668826.0,5749707.0,15954138.0,13741069.0,28622964.0,9393776.0,3998392.0,13392168.0,50000.0,15180796.0,15230796.0,35437416.0,3491275.0,2124877.0,3044086.0,618646.000,2.425439e+06,9.158,49.543,2.836159e+04,1.584800e+01,5.996000e+00,6.844,91.682,-4.780308e+06,9.400400e+01,3.510000e-01,0.924,3.710000e-01,8.792800e+01,1.887400e+01,5.321200e+01,1.144200e+01,6.167600e+01,2.625200e+01,3.779100e+01,1.273000e+00,4.920100e+01,6.387000e+00,7.718,11688.0,2.982126e+07,4.746110e+05,1.035680e+06,16.822722,안전기업,2021-12,1183.48044

In [ ]:
df_mean으로채운거.to_csv('평균으로크롤링채운거최종.csv',index=False)

## ESG 크롤링
- 아래 링크에서는 1005개의 기업, 2021년도만 뽑아올 수 있었다.
- baseline 데이터에 esg기업이 많지않아 크롤링 한것에만 의의를 두기로 했다.
- 컬럼 여부도 다 가져올것인지 esg 여부만 판단하는 컬럼만 추가할 것인지
- 비상장기업의 경우 지배구조만 갖고 판단한다고 한다.
- 링크 : http://www.cgs.or.kr/business/esg_tab04.jsp?&skey=&svalue=&sfyear=2021&styear=2021&sgtype=TOTAL&sgrade=#ui_contents

In [ ]:
esg_df = pd.read_html("http://www.cgs.or.kr/business/esg_tab04.jsp?pg=1&pp=1005&skey=&svalue=&sfyear=2021&styear=2021&sgtype=&sgrade=#ui_contents")[0]
esg_df.set_index('NO', inplace = True)
esg_df.sort_index(inplace=True)
# esg_df.to_csv('./ESG_DF.csv', encoding="utf-8-sig")

In [ ]:
esg_df

,기업명,기업코드 첨부내용,ESG등급,환경,사회,지배구조,평가년도,비고 (조정시기)
NO,,,,,,,,
1,흥아해운,3280,D,D,C,D,2021,-
2,흥국화재,540,B+,B,A,B,2021,-
3,흥국생명보험,818586,NaN,NaN,NaN,B,2021,-
4,휴켐스,69260,B+,B,A,B+,2021,-
5,휴젤,145020,B,D,B+,B+,2021,-
...,...,...,...,...,...,...,...,...
1001,BGF리테일,282330,A,A,A+,A,2021,-
1002,BGF,27410,A,A,A+,A,2021,-
1003,AP시스템,265520,D,D,D,C,2021,-


In [ ]:
len(list(esg_df['기업명'].values))
esg_name = list(esg_df['기업명'].values)

In [ ]:
tmp = []
for i in list(final_df['CMP_NM'].unique()):
  if i in esg_name:
    tmp.append(i)

In [ ]:
print(len(tmp))
print(tmp)

70
['태영건설', '현대코퍼레이션', '대웅', '한창', '신흥', '한미글로벌', '태경산업', '동화기업', '동서', '한진', '대덕', '성안', '우진', '동일산업', '광명전기', '동양', '진양산업', '일성건설', '화신', '유니켐', '이화산업', '효성', '동방', '동원산업', '오리온', '케이씨', '드림텍', '세원정공', '대현', '대동', '유니온', '에스엠', '대동전자', '태광산업', '폴루스바이오팜', '다스코', '에이스테크', '그린케미칼', '케이씨텍', '유진기업', '디아이씨', '웅진', '신원', '휴비스', '아이비케이연금보험', '대상', '엔케이', '국보', '에이엔피', '영풍', '서원', '한성기업', '에이아이에이생명보험', '세하', '우성사료', '위닉스', '녹십자셀', '삼부토건', '선도전기', '조흥', '컨버즈', '대아티아이', '경방', '와이즈버즈', '이스타코', '에이치엘비생명과학', '선진', '엔케이맥스', '화성산업', '신성이엔지']
